In [1]:
#--cell 1--#
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# For data analysis
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
#--cell 2--#
import json
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from datetime import datetime

class NBAOddsAndLineupsScraper:
    def __init__(self):
        self.session = requests.Session()
        self.setup_headers()
    
    def setup_headers(self):
        """Setup common headers for requests"""
        self.headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'accept-language': 'en-US,en;q=0.9',
            'cache-control': 'max-age=0',
            'priority': 'u=0, i',
            'referer': 'https://www.rotowire.com/',
            'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'document',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-site': 'same-origin',
            'sec-fetch-user': '?1',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'
        }

    # --------- RAW WIDE ODDS (no aggregation) ---------------------------------
    def get_player_props_odds_wide_raw(self, book='mgm'):
        """
        Return the raw 'wide' odds table by scraping Rotowire's player-props page.
        This preserves columns like:
          name, team, opp, <book>_pts, <book>_ptsUnder, <book>_ptsOver, ...
        Works across many books present in the page JSON blocks.
        """
        url = f"https://www.rotowire.com/betting/nba/player-props.php?book={book}"
        try:
            r = self.session.get(url, headers=self.headers)
            r.raise_for_status()
        except Exception as e:
            print(f"Failed to GET odds page: {e}")
            return pd.DataFrame()

        # Extract ALL JSON lists assigned to "data:" in the page
        matches = re.findall(r"data:\s*(\[\{.*?\}\])", r.text, flags=re.DOTALL)
        frames = []
        for m in matches:
            try:
                rows = json.loads(m)
                if isinstance(rows, list) and rows:
                    frames.append(pd.DataFrame(rows))
            except Exception:
                continue

        if not frames:
            print("No odds JSON blocks found.")
            return pd.DataFrame()

        df = pd.concat(frames, ignore_index=True)
        # keep the most useful id/basic columns if present
        base_cols = [c for c in ["name","gameID","playerID","firstName","lastName","team","opp","logo","playerLink"] if c in df.columns]
        other_cols = [c for c in df.columns if c not in base_cols]
        df = df[base_cols + other_cols]
        # normalize team/opponent field names
        if "opp" in df.columns and "opponent" not in df.columns:
            df = df.rename(columns={"opp": "opponent"})
        # add as-of date and (best-guess) game_date if not present
        df["asof_date"] = datetime.utcnow().strftime("%Y-%m-%d")
        if "game_date" not in df.columns:
            df["game_date"] = df["asof_date"]
        print(f"Successfully fetched RAW odds rows: {len(df)} | columns: {len(df.columns)}")
        return df

    # --------- Legacy aggregated method (kept in case you still call it) ------
    def get_player_props_odds(self, book='mgm'):
        """
        Old helper that aggregated rows by 'name'.
        Prefer get_player_props_odds_wide_raw() for modeling/joins.
        """
        wide = self.get_player_props_odds_wide_raw(book=book)
        if wide.empty:
            return None
        aggregated_df = wide.groupby('name', as_index=False, sort=False).agg(
            lambda x: ', '.join(pd.Series(x).dropna().astype(str).unique())
        )
        aggregated_df = aggregated_df.dropna(axis=1, how='all')
        print(f"Successfully aggregated odds for {len(aggregated_df)} players")
        return aggregated_df

    # --------- Lineups scraping (unchanged logic, made a bit sturdier) --------
    def get_expected_lineups(self):
        """Get expected lineups from Rotowire"""
        url = "https://www.rotowire.com/basketball/nba-lineups.php"
        try:
            r = self.session.get(url, headers=self.headers)
            r.raise_for_status()
        except Exception as e:
            print(f"Failed to retrieve lineup page: {e}")
            return None

        soup = BeautifulSoup(r.content, 'html.parser')
        game_containers = soup.find_all('div', class_='lineup__main')
        if not game_containers:
            print("No lineup data found")
            return None

        lineups_data = []
        for game in game_containers:
            game_info = self._parse_game_info(game)
            if game_info:
                lineups_data.append(game_info)

        print(f"Successfully fetched lineups for {len(lineups_data)} games")
        return lineups_data

    def _parse_game_info(self, game_container):
        """Parse individual game information and lineups"""
        try:
            game_data = {}
            header = game_container.find('div', class_='lineup__hdr')
            if header:
                teams = header.find_all('div', class_='lineup__team')
                if len(teams) >= 2:
                    game_data['away_team'] = teams[0].get_text(strip=True)
                    game_data['home_team'] = teams[1].get_text(strip=True)
            time_info = header.find('div', class_='lineup__time') if header else None
            if time_info:
                game_data['game_time'] = time_info.get_text(strip=True)
            lineup_containers = game_container.find_all('div', class_='lineup__box')
            if len(lineup_containers) >= 2:
                game_data['away_starters'] = self._parse_team_lineup(lineup_containers[0])
                game_data['home_starters'] = self._parse_team_lineup(lineup_containers[1])
            return game_data
        except Exception as e:
            print(f"Error parsing game info: {e}")
            return None

    def _parse_team_lineup(self, team_container):
        """Parse individual team lineup"""
        starters = []
        try:
            starters_section = team_container.find('div', class_='lineup__list')
            if starters_section:
                player_elements = starters_section.find_all('div', class_='lineup__player')
                for player_elem in player_elements:
                    player_info = self._parse_player_info(player_elem)
                    if player_info:
                        starters.append(player_info)
            return starters
        except Exception as e:
            print(f"Error parsing team lineup: {e}")
            return []

    def _parse_player_info(self, player_elem):
        """Parse individual player information"""
        try:
            player_data = {}
            name_elem = player_elem.find('a', class_='lineup__player-link')
            if name_elem:
                player_data['name'] = name_elem.get_text(strip=True)
                player_data['player_link'] = name_elem.get('href', '')
            pos_elem = player_elem.find('span', class_='lineup__pos')
            if pos_elem:
                player_data['position'] = pos_elem.get_text(strip=True)
            injury_elem = player_elem.find('span', class_='lineup__inj')
            player_data['injury_status'] = injury_elem.get_text(strip=True) if injury_elem else 'Active'
            confirmed_elem = player_elem.find('span', class_='lineup__confirm')
            player_data['confirmed_starter'] = confirmed_elem is not None
            return player_data
        except Exception as e:
            print(f"Error parsing player info: {e}")
            return None

    def get_comprehensive_data(self):
        """Get both odds and lineups data"""
        print("Fetching NBA betting data and lineups...")
        odds_data = self.get_player_props_odds_wide_raw()  # <-- use RAW wide
        lineups_data = self.get_expected_lineups()
        combined_data = {
            'odds': odds_data,
            'lineups': lineups_data,
            'last_updated': datetime.now().isoformat()
        }
        return combined_data
    
    def save_to_excel(self, data, filename=None):
        """Save the scraped data to Excel files"""
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f'nba_betting_data_{timestamp}.xlsx'
        try:
            with pd.ExcelWriter(filename, engine='openpyxl') as writer:
                if isinstance(data.get('odds'), pd.DataFrame) and not data['odds'].empty:
                    data['odds'].to_excel(writer, sheet_name='Player_Odds', index=False)
                if data.get('lineups') is not None:
                    lineups_list = []
                    for game in data['lineups']:
                        for starter_type in ['away_starters', 'home_starters']:
                            team = game.get('away_team' if starter_type == 'away_starters' else 'home_team', 'Unknown')
                            starters = game.get(starter_type, [])
                            for starter in starters:
                                lineups_list.append({
                                    'game_time': game.get('game_time', ''),
                                    'team': team,
                                    'player_name': starter.get('name', ''),
                                    'position': starter.get('position', ''),
                                    'injury_status': starter.get('injury_status', ''),
                                    'confirmed_starter': starter.get('confirmed_starter', False),
                                    'player_link': starter.get('player_link', '')
                                })
                    if lineups_list:
                        lineups_df = pd.DataFrame(lineups_list)
                        lineups_df.to_excel(writer, sheet_name='Expected_Lineups', index=False)
                metadata = pd.DataFrame([{
                    'last_updated': data.get('last_updated', ''),
                    'total_games': len(data.get('lineups', [])) if isinstance(data.get('lineups'), list) else 0,
                    'total_players_odds': len(data.get('odds', [])) if isinstance(data.get('odds'), pd.DataFrame) else 0
                }])
                metadata.to_excel(writer, sheet_name='Metadata', index=False)
            print(f"Data successfully saved to {filename}")
            return True
        except Exception as e:
            print(f"Error saving to Excel: {e}")
            return False

# Usage example and integration with your existing analytics
def integrate_with_analytics():
    """Integrate the scraper with your existing analytics"""
    scraper = NBAOddsAndLineupsScraper()
    nba_data = scraper.get_comprehensive_data()
    scraper.save_to_excel(nba_data)
    processed_data = process_for_analytics(nba_data)
    return processed_data

def process_for_analytics(nba_data):
    """Process the scraped data for use in analytics"""
    processed = {}
    # Odds data → extract basic lines for PTS/REB/AST if present
    if isinstance(nba_data.get('odds'), pd.DataFrame) and not nba_data['odds'].empty:
        odds_df = nba_data['odds']
        def pick_line(row, market):
            # Look for any <book>_<marketLower> columns (line, Under, Over)
            m = market.lower()
            line = None
            over = None
            under = None
            for col in row.index:
                c = col.lower()
                if c.endswith(f"_{m}"):
                    line = row[col]
                elif c.endswith(f"_{m}over"):
                    over = row[col]
                elif c.endswith(f"_{m}under"):
                    under = row[col]
            try:
                line = float(line) if line is not None and str(line).strip() not in ("", "None", "nan") else None
            except Exception:
                line = None
            return line, over, under

        betting_lines = []
        for _, r in odds_df.iterrows():
            player_name = r.get('name', '')
            for mk in ["pts", "reb", "ast"]:
                line, over, under = pick_line(r, mk)
                if line is not None:
                    betting_lines.append({
                        "player": player_name,
                        "stat": {"pts":"points","reb":"rebounds","ast":"assists"}[mk],
                        "line": line,
                        "over_odds": over,
                        "under_odds": under
                    })
        processed['betting_lines'] = pd.DataFrame(betting_lines)

    # Lineups
    if nba_data.get('lineups') is not None:
        lineups = nba_data['lineups']
        team_players = {}
        for game in lineups:
            away_team = game.get('away_team')
            if away_team and away_team not in team_players:
                team_players[away_team] = []
            for starter in game.get('away_starters', []):
                if away_team and starter.get('name'):
                    team_players[away_team].append({
                        'name': starter['name'],
                        'position': starter.get('position', ''),
                        'status': starter.get('injury_status', 'Active'),
                        'confirmed': starter.get('confirmed_starter', False)
                    })
            home_team = game.get('home_team')
            if home_team and home_team not in team_players:
                team_players[home_team] = []
            for starter in game.get('home_starters', []):
                if home_team and starter.get('name'):
                    team_players[home_team].append({
                        'name': starter['name'],
                        'position': starter.get('position', ''),
                        'status': starter.get('injury_status', 'Active'),
                        'confirmed': starter.get('confirmed_starter', False)
                    })
        processed['team_lineups'] = team_players
        processed['games_today'] = lineups
    return processed

def extract_betting_line(player_row, stat_type):
    """Extract betting line for specific stat type (legacy helper)"""
    line_col = over_odds_col = under_odds_col = None
    for col in player_row.index:
        col_lower = col.lower()
        if stat_type in col_lower and 'line' in col_lower:
            line_col = col
        elif stat_type in col_lower and 'over' in col_lower and 'odds' in col_lower:
            over_odds_col = col
        elif stat_type in col_lower and 'under' in col_lower and 'odds' in col_lower:
            under_odds_col = col
    line_value = player_row.get(line_col) if line_col else None
    if line_value and str(line_value).replace('.', '').isdigit():
        return {
            'line': float(line_value),
            'over_odds': player_row.get(over_odds_col) if over_odds_col else None,
            'under_odds': player_row.get(under_odds_col) if under_odds_col else None
        }
    return None

# Main execution
if __name__ == "__main__":
    data = integrate_with_analytics()
    print("\n" + "="*50)
    print("NBA BETTING DATA SUMMARY")
    print("="*50)
    if data.get('betting_lines') is not None:
        print(f"\nBetting Lines: {len(data['betting_lines'])} player-stat combinations")
        print(data['betting_lines'].head(10))
    if data.get('team_lineups'):
        print(f"\nTeams with Lineups: {len(data['team_lineups'])}")
        for team, players in list(data['team_lineups'].items())[:3]:
            print(f"{team}: {len(players)} players")
            for player in players[:3]:
                print(f"  - {player['name']} ({player['position']}) - {player['status']}")
    if data.get('games_today'):
        print(f"\nGames Today: {len(data['games_today'])}")
        for game in data['games_today'][:3]:
            print(f"{game.get('away_team', 'TBD')} @ {game.get('home_team', 'TBD')} - {game.get('game_time', 'Time TBD')}")


Fetching NBA betting data and lineups...
Successfully fetched RAW odds rows: 1850 | columns: 263
Successfully fetched lineups for 0 games
Data successfully saved to nba_betting_data_20251104_002811.xlsx

NBA BETTING DATA SUMMARY

Betting Lines: 0 player-stat combinations
Empty DataFrame
Columns: []
Index: []


In [3]:
#--cell 3--#
def get_daily_matchups(date=None):
    """Get NBA games for a specific date"""
    if date is None:
        date = datetime.now().strftime('%Y-%m-%d')
    # Placeholder demo; replace with a real schedule API if desired
    sample_matchups = [
        {'home_team': 'GSW', 'away_team': 'LAL', 'time': '7:30 PM ET'},
        {'home_team': 'BOS', 'away_team': 'MIA', 'time': '8:00 PM ET'},
        {'home_team': 'DEN', 'away_team': 'DAL', 'time': '9:00 PM ET'},
    ]
    return sample_matchups

def calculate_player_correlations(player_a_logs, player_b_logs):
    """Calculate correlation between two players' performances"""
    merged = pd.merge(player_a_logs, player_b_logs, on='GAME_DATE', suffixes=('_a', '_b'))
    correlations = {}
    for stat in ['PTS', 'REB', 'AST']:
        if f'{stat}_a' in merged.columns and f'{stat}_b' in merged.columns:
            corr = merged[f'{stat}_a'].corr(merged[f'{stat}_b'])
            correlations[stat] = corr
    return correlations

# Export results to Excel
def export_analysis(results, filename='nba_betting_analysis.xlsx'):
    """Export analysis results to Excel"""
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        if 'value_bets' in results:
            pd.DataFrame(results['value_bets']).to_excel(writer, sheet_name='Value_Bets', index=False)
        if 'predictions' in results:
            predictions_df = pd.DataFrame.from_dict(results['predictions'], orient='index')
            predictions_df.to_excel(writer, sheet_name='Player_Predictions')
    print(f"Analysis exported to {filename}")


In [4]:
#--cell 4--#
import requests
import pandas as pd
import time
import random

url = "https://stats.nba.com/stats/leaguedashplayerstats"

base_params = {
    "College": "",
    "Conference": "",
    "Country": "",
    "DateFrom": "",
    "DateTo": "",
    "Division": "",
    "DraftPick": "",
    "DraftYear": "",
    "GameScope": "",
    "GameSegment": "",
    "Height": "",
    "ISTRound": "",
    "LastNGames": "0",
    "LeagueID": "00",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "",
    "PlusMinus": "N",
    "Rank": "N",
    "SeasonSegment": "",
    "SeasonType": "Regular Season",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "VsConference": "",
    "VsDivision": "",
    "Weight": ""
}

headers = {
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en;q=0.9",
    "Origin": "https://www.nba.com",
    "Referer": "https://www.nba.com/",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/141.0.0.0 Safari/537.36",
    "x-nba-stats-origin": "stats",
    "x-nba-stats-token": "true"
}

seasons = ["2023-24", "2024-25"]

def fetch_season_data(season, retries=3):
    """Fetch one season’s player stats, retrying if timeout or network error."""
    params = base_params.copy()
    params["Season"] = season

    for attempt in range(1, retries + 1):
        try:
            print(f"→ Attempt {attempt} fetching {season} data...")
            response = requests.get(url, headers=headers, params=params, timeout=30)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.Timeout:
            print(f"⚠️ Timeout on attempt {attempt}/{retries} for {season}. Retrying...")
            time.sleep(3 * attempt)
        except requests.exceptions.RequestException as e:
            print(f"❌ Error on attempt {attempt}/{retries}: {e}")
            time.sleep(3 * attempt)
    raise RuntimeError(f"Failed to fetch {season} data after {retries} attempts.")

# Main loop
for season in seasons:
    print(f"\n🏀 Fetching NBA stats for {season}...")
    data = fetch_season_data(season)

    headers_list = data["resultSets"][0]["headers"]
    rows = data["resultSets"][0]["rowSet"]

    df = pd.DataFrame(rows, columns=headers_list)
    filename = f"nba_player_stats_{season.replace('-', '_')}.csv"
    df.to_csv(filename, index=False)

    print(f"✅ {season}: saved {len(df)} player records to '{filename}'")

    # Wait 3–6 seconds before next season to avoid throttling
    time.sleep(random.uniform(3, 6))

print("\n🎉 Done! Both 2023-24 and 2024-25 seasons downloaded.")



🏀 Fetching NBA stats for 2023-24...
→ Attempt 1 fetching 2023-24 data...
✅ 2023-24: saved 572 player records to 'nba_player_stats_2023_24.csv'

🏀 Fetching NBA stats for 2024-25...
→ Attempt 1 fetching 2024-25 data...
✅ 2024-25: saved 569 player records to 'nba_player_stats_2024_25.csv'

🎉 Done! Both 2023-24 and 2024-25 seasons downloaded.


In [5]:
#--cell 5--#
import requests
import pandas as pd
import time

def get_box_scores(season, season_type="Regular Season"):
    url = "https://stats.nba.com/stats/leaguegamelog"
    params = {
        "Counter": 1000,
        "DateFrom": "",
        "DateTo": "",
        "Direction": "DESC",
        "ISTRound": "",
        "LeagueID": "00",
        "PlayerOrTeam": "P",
        "Season": season,
        "SeasonType": season_type,
        "Sorter": "DATE"
    }
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36",
        "Referer": "https://www.nba.com/",
        "Origin": "https://www.nba.com",
        "Accept": "application/json, text/plain, */*"
    }

    response = requests.get(url, params=params, headers=headers, timeout=30)
    response.raise_for_status()

    data = response.json()["resultSets"][0]
    df = pd.DataFrame(data["rowSet"], columns=data["headers"])
    return df

# Get all three seasons
seasons = ["2023-24", "2024-25", "2025-26"]
for season in seasons:
    print(f"Fetching {season}...")
    df = get_box_scores(season)
    df.to_csv(f"nba_boxscores_{season}.csv", index=False)
    print(f"✅ Saved {len(df)} records for {season}")
    time.sleep(2)  # polite delay


Fetching 2023-24...
✅ Saved 26401 records for 2023-24
Fetching 2024-25...
✅ Saved 26306 records for 2024-25
Fetching 2025-26...
✅ Saved 2115 records for 2025-26


In [6]:
#--cell 6--#
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd
import requests
import io
import unicodedata

# ---- Keep/Map settings -------------------------------------------------------

ADV_COLS_KEEP = [
    "Player", "Pos", "Age", "Tm", "G", "MP",
    "PER", "TS%", "3PAr", "FTr",
    "ORB%", "DRB%", "TRB%",
    "AST%", "STL%", "BLK%",
    "TOV%", "USG%",
    "ORtg", "DRtg",
    "OWS", "DWS", "WS", "WS/48",
    "OBPM", "DBPM", "BPM", "VORP"
]

# Basketball-Reference -> NBA/your dataset codes
TEAM_ABBR_MAP = {
    "BRK": "BKN",
    "PHO": "PHX",
    "CHO": "CHA",
    "UTH": "UTA",   # rare alias safety
    "NJN": "BKN",   # historical
    "SEA": "OKC",   # historical
    "VAN": "MEM",   # historical
}

# ---- Helpers -----------------------------------------------------------------

def normalize_name(s):
    """Normalize player names for consistent joining (lowercase, no accents/punct)."""
    if pd.isna(s):
        return s
    s = s.strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    for ch in [".", "'", "`", "’", "“", "”", ","]:
        s = s.replace(ch, "")
    s = " ".join(s.split())
    return s

# ---- Fetch advanced table from Basketball-Reference --------------------------

def fetch_advanced_table(season=2026):
    """
    Fetch and clean Basketball-Reference advanced stats table for a given season.
    Example: season=2025 -> https://www.basketball-reference.com/leagues/NBA_2025_advanced.html
    """
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_advanced.html"
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        ),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    }
    resp = requests.get(url, headers=headers, timeout=30)
    resp.raise_for_status()

    tables = pd.read_html(io.StringIO(resp.text), header=0)
    if not tables:
        raise RuntimeError("No tables found on Basketball-Reference page.")

    df = tables[0].copy()

    # Remove duplicate header rows
    if "Rk" in df.columns:
        df = df[df["Rk"] != "Rk"].copy()
        df.drop(columns=["Rk"], inplace=True, errors="ignore")

    # Normalize column names (strip and upper-case for easy access)
    df.columns = [c.strip() for c in df.columns]

    # Basketball Reference sometimes labels the team column differently — make sure it exists
    team_col = None
    for c in df.columns:
        if c.lower() in ["tm", "team", "team_name"]:
            team_col = c
            break
    if not team_col:
        raise KeyError(f"Could not find a team column in advanced table. Found: {df.columns.tolist()}")
    df.rename(columns={team_col: "Tm"}, inplace=True)

    # Keep relevant columns if present
    keep = [c for c in ADV_COLS_KEEP if c in df.columns]
    df = df[keep].copy()

    # Convert numeric columns
    non_numeric = {"Player", "Pos", "Tm"}
    for c in [c for c in df.columns if c not in non_numeric]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # Map team abbreviations to match your dataset
    df["Tm"] = df["Tm"].replace(TEAM_ABBR_MAP)

    # Add join keys
    df["player_key"] = df["Player"].map(normalize_name)
    df["team_key"] = df["Tm"].astype(str).str.strip().str.upper()

    return df

# ---- Load your averages CSV and align columns --------------------------------

def load_averages_csv(path):
    """
    Load your NBA averages CSV (with headers like PLAYER_NAME, TEAM_ABBREVIATION).
    Renames to canonical 'Player' and 'Team' and adds join keys.
    """
    df = pd.read_csv(path)

    # Auto-map your headers to canonical names
    col_map = {}
    for c in df.columns:
        cl = c.strip().lower()
        if cl == "player_name":
            col_map[c] = "Player"
        elif cl in ("team_abbreviation", "tm", "team"):
            col_map[c] = "Team"
        # keep other columns as-is

    df = df.rename(columns=col_map)

    if "Player" not in df.columns or "Team" not in df.columns:
        raise ValueError(
            "Couldn't find columns for 'Player' and 'Team'. "
            f"Available columns: {list(df.columns)}"
        )

    # Join keys
    df["player_key"] = df["Player"].map(normalize_name)
    df["team_key"] = df["Team"].astype(str).str.strip().str.upper()

    return df

# ---- Merge logic (with TOT fallback for traded players) ----------------------

def merge_advanced_into_averages(df_avg, df_adv):
    """
    Merge advanced metrics into averages.
    1) Exact Player+Team match (ignore TOT).
    2) For remaining NaNs, fill from TOT row by Player.
    """
    adv_team = df_adv[df_adv["Tm"] != "TOT"].copy()
    adv_tot  = df_adv[df_adv["Tm"] == "TOT"].copy()

    adv_cols_to_add = [c for c in df_adv.columns if c not in {"Player", "Pos", "Age", "Tm", "player_key", "team_key"}]
    meta_cols = [c for c in ["Pos", "Age"] if c in df_adv.columns]
    join_cols_full = meta_cols + adv_cols_to_add

    merged = df_avg.merge(
        adv_team[["player_key", "team_key"] + join_cols_full],
        on=["player_key", "team_key"],
        how="left",
        suffixes=("", "_adv"),
    )

    # Determine "missing" based on a representative advanced column
    probe_col = "PER" if "PER" in merged.columns else ("WS/48" if "WS/48" in merged.columns else None)
    missing_mask = merged[probe_col].isna() if probe_col else merged.isna().any(axis=1)

    if missing_mask.any() and not adv_tot.empty:
        fallback = merged[missing_mask].merge(
            adv_tot[["player_key"] + join_cols_full],
            on="player_key",
            how="left",
            suffixes=("", "_tot"),
        )
        for col in join_cols_full:
            if col in merged.columns and col in fallback.columns:
                merged.loc[missing_mask, col] = merged.loc[missing_mask, col].fillna(fallback[col])

    return merged

# ==============================================================================
# Example usage for your two files
# ==============================================================================

# ---- 2023–24 (Basketball-Reference season code = 2024) -----------------------
df_avg_2024 = load_averages_csv("nba_player_stats_2023_24.csv")
df_adv_2024 = fetch_advanced_table(season=2024)
df_enriched_2024 = merge_advanced_into_averages(df_avg_2024, df_adv_2024)
df_enriched_2024.to_csv("nba_player_stats_2023_24_enriched.csv", index=False)
print("✅ Saved: nba_player_stats_2023_24_enriched.csv")

# ---- 2024–25 (Basketball-Reference season code = 2025) -----------------------
df_avg_2025 = load_averages_csv("nba_player_stats_2024_25.csv")
df_adv_2025 = fetch_advanced_table(season=2025)
df_enriched_2025 = merge_advanced_into_averages(df_avg_2025, df_adv_2025)
df_enriched_2025.to_csv("nba_player_stats_2024_25_enriched.csv", index=False)
print("✅ Saved: nba_player_stats_2024_25_enriched.csv")

# ---- (Optional) Combine both seasons into one file ---------------------------
df_combined = pd.concat([df_enriched_2024, df_enriched_2025], ignore_index=True)
df_combined.to_csv("nba_player_stats_2023_25_combined.csv", index=False)
print("🏀 Combined: nba_player_stats_2023_25_combined.csv")


✅ Saved: nba_player_stats_2023_24_enriched.csv
✅ Saved: nba_player_stats_2024_25_enriched.csv
🏀 Combined: nba_player_stats_2023_25_combined.csv


In [7]:
# -- CELL 7a: assemble_player_game_features helper ----------------------------
import pandas as pd
import numpy as np

def assemble_player_game_features(logs_df: pd.DataFrame, enriched_df: pd.DataFrame) -> pd.DataFrame:
    """
    Combine raw boxscore logs + per-player enriched season stats into a
    training-ready feature table.

    Output columns include:
      • GAME_DATE, PLAYER_ID, PLAYER_NAME, TEAM_ABBREVIATION, OPPONENT_ABBREVIATION
      • rolling stats (PTS_roll5, REB_roll5, etc.)
      • efficiency metrics from enriched_df (PER, TS%, USG%, BPM, VORP, ORtg, DRtg, WS/48)
      • rest / HOME indicators and next-game targets (PTS_next, REB_next, AST_next)
    """

    df = logs_df.copy()
    df["GAME_DATE"] = pd.to_datetime(df["GAME_DATE"], errors="coerce")

    # sort for proper rolling windows
    df = df.sort_values(["PLAYER_ID", "GAME_DATE"]).reset_index(drop=True)

    # numeric ensure
    for col in ["PTS","REB","AST","MIN"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # rolling means (momentum features)
    for stat in ["PTS","REB","AST","MIN"]:
        df[f"{stat}_roll5"]  = (df.groupby("PLAYER_ID")[stat]
                                  .rolling(5, min_periods=1).mean().reset_index(level=0, drop=True))
        df[f"{stat}_roll10"] = (df.groupby("PLAYER_ID")[stat]
                                  .rolling(10, min_periods=1).mean().reset_index(level=0, drop=True))

    # next-game “target” columns
    for stat in ["PTS","REB","AST"]:
        df[f"{stat}_next"] = df.groupby("PLAYER_ID")[stat].shift(-1)

    # days_rest
    if "GAME_DATE" in df.columns:
        df["days_rest"] = (df.groupby("PLAYER_ID")["GAME_DATE"]
                             .diff().dt.days.clip(lower=0).fillna(3))

    # home/away flag (if not already encoded)
    if "MATCHUP" in df.columns:
        df["HOME"] = (~df["MATCHUP"].str.contains("@")).astype(int)

    # merge in season-level efficiency stats
    common_key = "PLAYER_ID" if "PLAYER_ID" in enriched_df.columns else "PLAYER_NAME"
    eff_cols = ["PER","TS%","USG%","BPM","VORP","ORtg","DRtg","WS/48"]
    eff_cols = [c for c in eff_cols if c in enriched_df.columns]
    df = df.merge(enriched_df[[common_key]+eff_cols].drop_duplicates(common_key),
                  on=common_key, how="left")

    # clip out extreme or empty records
    df = df.dropna(subset=["PTS","REB","AST","MIN"])
    df = df[df["MIN"] > 0]

    print(f"✅ Built features for {df['PLAYER_ID'].nunique()} players — {len(df):,} rows.")
    return df


In [ ]:
# -- CELL 8 (Final Robust Version) --------------------------------------------
# XGBoost player models with: player-wise CV, feature engineering, opponent context,
# optional hyperparameter tuning, and quantile models for uncertainty.

import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from collections import defaultdict

# ----------------------------
# 0) Load season data
# ----------------------------
logs_2324      = pd.read_csv("nba_boxscores_2023-24.csv")
logs_2425      = pd.read_csv("nba_boxscores_2024-25.csv")
enriched_2324  = pd.read_csv("nba_player_stats_2023_24_enriched.csv")
enriched_2425  = pd.read_csv("nba_player_stats_2024_25_enriched.csv")

feat_2324 = assemble_player_game_features(logs_2324, enriched_2324)
feat_2425 = assemble_player_game_features(logs_2425, enriched_2425)
features_all = pd.concat([feat_2324, feat_2425], ignore_index=True)

if "GAME_DATE" in features_all.columns:
    features_all["GAME_DATE"] = pd.to_datetime(features_all["GAME_DATE"])

# ----------------------------
# 1) Feature Engineering
# ----------------------------
def safe_has(cols): 
    return all(c in features_all.columns for c in cols)

# Trend (momentum) features
for stat in ["PTS", "REB", "AST"]:
    c5, c10 = f"{stat}_roll5", f"{stat}_roll10"
    if safe_has([c5, c10]):
        features_all[f"{stat}_trend"] = features_all[c5] - features_all[c10]

# Rolling volatility
def ensure_roll5_std(stat):
    col_std = f"{stat}_roll5_std"
    if col_std not in features_all.columns and stat in features_all.columns:
        features_all[col_std] = (
            features_all.sort_values(["PLAYER_ID","GAME_DATE"])
                        .groupby("PLAYER_ID")[stat]
                        .rolling(5, min_periods=1).std()
                        .reset_index(level=0, drop=True)
        )
for stat in ["PTS","REB","AST"]:
    ensure_roll5_std(stat)

# Interactions
if safe_has(["USG%","MIN_roll5"]):
    features_all["usage_minutes_interact"] = features_all["USG%"] * features_all["MIN_roll5"]
if safe_has(["TS%","USG%"]):
    features_all["ts_usage_interact"] = features_all["TS%"] * features_all["USG%"]

# Back-to-back indicator
if "days_rest" in features_all.columns and "is_b2b" not in features_all.columns:
    features_all["is_b2b"] = (features_all["days_rest"] == 0).astype(int)

# ensure boolean-like columns are numeric
for col in ["HOME", "is_b2b"]:
    if col in features_all.columns and features_all[col].dtype == bool:
        features_all[col] = features_all[col].astype(int)

# ----------------------------
# 2) Opponent / Matchup Context
# ----------------------------
def build_team_context(df: pd.DataFrame) -> pd.DataFrame:
    def _dedupe_by_name(d: pd.DataFrame, names=("TEAM_ABBREVIATION","OPPONENT_ABBREVIATION","GAME_DATE")) -> pd.DataFrame:
        d = d.copy()
        for nm in names:
            idxs = [i for i, c in enumerate(d.columns) if c == nm]
            if len(idxs) > 1:
                to_drop = [d.columns[i] for i in idxs[1:]]
                d.drop(columns=to_drop, inplace=True, errors="ignore")
                print(f"🧹 Deduped duplicate column '{nm}' (dropped {len(to_drop)} duplicates).")
        if d.columns.duplicated().any():
            d = d.loc[:, ~d.columns.duplicated()].copy()
        return d

    req = ["GAME_DATE", "TEAM_ABBREVIATION", "OPPONENT_ABBREVIATION"]
    ctx_cols = [c for c in ["ORtg_g_roll5", "DRtg_g_roll5", "Pace_g_roll5"] if c in df.columns]
    if not all(c in df.columns for c in req) or not ctx_cols:
        print("⚠️ Skipping team context: missing required columns.")
        return df

    df_clean = _dedupe_by_name(df)
    team_day = (
        df_clean[req + ctx_cols]
        .drop_duplicates(subset=["GAME_DATE", "TEAM_ABBREVIATION"])
        .groupby(["GAME_DATE", "TEAM_ABBREVIATION"], as_index=False)
        .mean(numeric_only=True)
    )
    opp_day = team_day.rename(
        columns={"TEAM_ABBREVIATION": "OPPONENT_ABBREVIATION", **{c: f"opp_{c}" for c in ctx_cols}}
    )
    opp_day = _dedupe_by_name(opp_day, names=("GAME_DATE","OPPONENT_ABBREVIATION"))
    right_cols = ["GAME_DATE","OPPONENT_ABBREVIATION"] + [f"opp_{c}" for c in ctx_cols]
    opp_day = opp_day[right_cols].drop_duplicates(subset=["GAME_DATE","OPPONENT_ABBREVIATION"])
    out = df_clean.merge(opp_day, on=["GAME_DATE","OPPONENT_ABBREVIATION"], how="left", validate="m:1")
    if "Pace_g_roll5" in out.columns and "opp_Pace_g_roll5" in out.columns:
        out["pace_diff5"] = out["Pace_g_roll5"] - out["opp_Pace_g_roll5"]
    print(f"✅ Team context added. Rows: {len(out):,}")
    return out

features_all = build_team_context(features_all)

# ----------------------------
# 3) Training setup
# ----------------------------
BASE_FEATURES = [
    "MIN_roll5","MIN_roll10","TS_game_roll5","TS_game_roll10",
    "usage_share_roll5","ORtg_g_roll5","DRtg_g_roll5","Pace_g_roll5",
    "PER","TS%","USG%","ORtg","DRtg","WS/48","BPM","VORP",
    "days_rest","is_b2b","HOME",
    "PTS_trend","REB_trend","AST_trend",
    "PTS_roll5_std","REB_roll5_std","AST_roll5_std",
    "usage_minutes_interact","ts_usage_interact",
    "opp_ORtg_g_roll5","opp_DRtg_g_roll5","opp_Pace_g_roll5","pace_diff5"
]
STAT_ROLL = {"PTS":["PTS_roll5","PTS_roll10"],"REB":["REB_roll5","REB_roll10"],"AST":["AST_roll5","AST_roll10"]}
TARGETS = {"PTS":"PTS_next","REB":"REB_next","AST":"AST_next"}

USE_OPTUNA = False
DEFAULT_SPLITS = 5

models_mean, models_q_lo, models_q_hi = {}, {}, {}
cv_scores, feature_bags = {}, {}

# ----------------------------
# 4) Training loop
# ----------------------------
for stat, target_col in TARGETS.items():
    feat_cols = [c for c in (BASE_FEATURES + STAT_ROLL[stat]) if c in features_all.columns]
    feature_bags[stat] = feat_cols
    needed = feat_cols + [target_col,"PLAYER_ID","GAME_DATE"]
    data = features_all.dropna(subset=[c for c in needed if c in features_all.columns]).copy()
    if data.empty:
        print(f"⚠️ No data for {stat}.")
        continue

    data = data.sort_values(["PLAYER_ID","GAME_DATE"])
    X, y, groups = data[feat_cols], data[target_col], data["PLAYER_ID"]

    unique_players = groups.nunique()
    if unique_players < 2:
        print(f"⚠️ Only one player for {stat}; fallback to simple split.")
        cut = int(len(X)*0.8)
        Xtr, Xte, ytr, yte = X.iloc[:cut], X.iloc[cut:], y.iloc[:cut], y.iloc[cut:]
        model = XGBRegressor(n_estimators=400,learning_rate=0.05,max_depth=5,
                             subsample=0.85,colsample_bytree=0.9,
                             reg_lambda=1.0,reg_alpha=0.0,
                             random_state=42,n_jobs=-1,verbosity=0)
        model.fit(Xtr,ytr)
        pred = model.predict(Xte)
        mae = mean_absolute_error(yte,pred)
        fold_mae=[mae]
    else:
        splits = min(DEFAULT_SPLITS, max(2, unique_players))
        if splits < DEFAULT_SPLITS:
            print(f"ℹ️ Reduced splits for {stat} to {splits} (players={unique_players}).")
        group_kfold = GroupKFold(n_splits=splits)
        fold_mae=[]
        for tr,te in group_kfold.split(X,y,groups):
            model=XGBRegressor(n_estimators=400,learning_rate=0.05,max_depth=5,
                               subsample=0.85,colsample_bytree=0.9,
                               reg_lambda=1.0,reg_alpha=0.0,
                               random_state=42,n_jobs=-1,verbosity=0)
            model.fit(X.iloc[tr],y.iloc[tr])
            pred=model.predict(X.iloc[te])
            fold_mae.append(mean_absolute_error(y.iloc[te],pred))

    cv_scores[stat]=(float(np.mean(fold_mae)),float(np.std(fold_mae)))
    print(f"🔁 {stat} MAE: {np.mean(fold_mae):.2f} ± {np.std(fold_mae):.2f}")

    model_mean=XGBRegressor(n_estimators=400,learning_rate=0.05,max_depth=5,
                             subsample=0.85,colsample_bytree=0.9,
                             reg_lambda=1.0,reg_alpha=0.0,
                             random_state=42,n_jobs=-1,verbosity=0)
    model_mean.fit(X,y)
    models_mean[stat]=model_mean

    # Quantile fits (if supported)
    try:
        q_lo=XGBRegressor(objective="reg:pquantile",alpha=0.2,**model_mean.get_xgb_params())
        q_hi=XGBRegressor(objective="reg:pquantile",alpha=0.8,**model_mean.get_xgb_params())
        q_lo.fit(X,y); q_hi.fit(X,y)
        models_q_lo[stat],models_q_hi[stat]=q_lo,q_hi
    except Exception as e:
        print(f"⚠️ Quantile model failed for {stat}: {e}")

# ----------------------------
# 5) Importances
# ----------------------------
def top_importances(model, cols, k=15):
    try: gain=model.get_booster().get_score(importance_type="gain")
    except Exception: gain=model.get_booster().get_fscore()
    if not gain: return pd.Series(dtype=float)
    name_map={f"f{i}":col for i,col in enumerate(cols)}
    s=pd.Series(gain).rename(index=lambda x:name_map.get(x,x))
    s=pd.to_numeric(s,errors="coerce")
    return s.sort_values(ascending=False).head(k)

for stat in models_mean:
    print(f"\nTop {stat} importances:")
    imp=top_importances(models_mean[stat],feature_bags[stat])
    print(imp if not imp.empty else "(none)")

# ----------------------------
# 6) Game-day projection helper
# ----------------------------
def project_players(df_today: pd.DataFrame) -> pd.DataFrame:
    if df_today is None or df_today.empty:
        return pd.DataFrame()
    frames=[]
    for stat in ["PTS","REB","AST"]:
        if stat not in models_mean: continue
        cols=[c for c in feature_bags[stat] if c in df_today.columns]
        if not cols: continue
        X=df_today[cols]
        mu=models_mean[stat].predict(X)
        if stat in models_q_lo and stat in models_q_hi:
            try:
                ql=models_q_lo[stat].predict(X)
                qh=models_q_hi[stat].predict(X)
                sd=(qh-ql)/2.0
            except Exception: sd=np.full_like(mu,np.nan)
        else: sd=np.full_like(mu,np.nan)
        frames.append(pd.DataFrame({
            "PLAYER_ID":df_today.get("PLAYER_ID"),
            "PLAYER_NAME":df_today.get("PLAYER_NAME"),
            "TEAM_ABBREVIATION":df_today.get("TEAM_ABBREVIATION"),
            "OPPONENT_ABBREVIATION":df_today.get("OPPONENT_ABBREVIATION"),
            "market":stat,"projection_mean":mu,"projection_sd":sd
        }))
    return pd.concat(frames,ignore_index=True) if frames else pd.DataFrame()

print("\n✅ Training complete. Use `project_players(features_today)` to get mean & sd for PTS/REB/AST.")
print("Cross-validated MAE by stat:", cv_scores)


✅ Built features for 571 players — 26,283 rows.
✅ Built features for 569 players — 26,206 rows.
⚠️ Skipping team context: missing required columns.
🔁 PTS MAE: 4.55 ± 0.13
⚠️ Quantile model failed for PTS: xgboost.sklearn.XGBRegressor() got multiple values for keyword argument 'objective'
🔁 REB MAE: 1.93 ± 0.05
⚠️ Quantile model failed for REB: xgboost.sklearn.XGBRegressor() got multiple values for keyword argument 'objective'
🔁 AST MAE: 1.38 ± 0.05
⚠️ Quantile model failed for AST: xgboost.sklearn.XGBRegressor() got multiple values for keyword argument 'objective'

Top PTS importances:
usage_minutes_interact    17617.062500
PTS_roll10                 5413.078125
ts_usage_interact          1549.593506
TS%                         434.394745
WS/48                       368.271759
PER                         335.622955
MIN_roll10                  330.547943
PTS_roll5                   314.593933
VORP                        294.220703
USG%                        273.860565
BPM              

In [9]:
# -- CELL 9 (updated for new model structure) ---------------------------------
import pandas as pd

# Detect which model dictionary is available
if 'models_mean' in globals() and models_mean:
    active_models = models_mean
elif 'models' in globals() and models:
    active_models = models
else:
    raise RuntimeError("No trained models found. Run Cell 8 first.")

all_imps = {}

for stat, mdl in active_models.items():
    # Try to find associated feature columns
    if 'feature_bags' in globals():
        feat_cols = feature_bags.get(stat, [])
    elif 'feature_cols_by_stat' in globals():
        feat_cols = feature_cols_by_stat.get(stat, [])
    else:
        feat_cols = []

    # Importance extraction
    if hasattr(mdl, "feature_importances_"):
        imp_series = pd.Series(mdl.feature_importances_, index=feat_cols).sort_values(ascending=False)
    else:
        try:
            gain = mdl.get_booster().get_score(importance_type="gain")
            name_map = {f"f{i}": col for i, col in enumerate(feat_cols)}
            imp_series = pd.Series(gain).rename(index=lambda x: name_map.get(x, x)).astype(float)
            imp_series = imp_series.sort_values(ascending=False)
        except Exception:
            imp_series = pd.Series(dtype=float)

    all_imps[stat] = imp_series

    print(f"\nTop 15 importances — {stat}:")
    if not imp_series.empty:
        display(imp_series.head(15))
    else:
        print("(no importances available)")

# For backward compatibility (PTS importances stored as imp)
if "PTS" in all_imps:
    imp = all_imps["PTS"]

print("\n✅ Feature importance summaries computed for:", list(all_imps.keys()))



Top 15 importances — PTS:


usage_minutes_interact    0.602913
PTS_roll10                0.185253
ts_usage_interact         0.053032
TS%                       0.014866
WS/48                     0.012603
PER                       0.011486
MIN_roll10                0.011312
PTS_roll5                 0.010766
VORP                      0.010069
USG%                      0.009372
BPM                       0.008908
MIN_roll5                 0.008580
PTS_roll5_std             0.008302
AST_roll5_std             0.008180
REB_roll5_std             0.008144
dtype: float32


Top 15 importances — REB:


REB_roll10                0.658034
REB_roll5                 0.050810
WS/48                     0.023472
PER                       0.023164
usage_minutes_interact    0.018480
VORP                      0.017809
BPM                       0.017239
MIN_roll5                 0.016696
ts_usage_interact         0.016686
TS%                       0.016047
MIN_roll10                0.015408
REB_roll5_std             0.014891
AST_roll5_std             0.014588
REB_trend                 0.014353
AST_trend                 0.014298
dtype: float32


Top 15 importances — AST:


AST_roll10                0.692874
AST_roll5                 0.047897
ts_usage_interact         0.020068
VORP                      0.019310
USG%                      0.018214
usage_minutes_interact    0.017467
WS/48                     0.016049
TS%                       0.015541
BPM                       0.014501
REB_roll5_std             0.014000
PER                       0.013582
PTS_roll5_std             0.013439
MIN_roll5                 0.013361
AST_roll5_std             0.012854
AST_trend                 0.012679
dtype: float32


✅ Feature importance summaries computed for: ['PTS', 'REB', 'AST']


In [12]:
# -- CELL 10 (final robust version) ------------------------------------------
import pandas as pd
from sklearn.linear_model import Ridge

# 1️⃣ Try to ensure team-level rolling stats exist, or fall back to simpler proxies
context_bases = []
for base in ["ORtg", "DRtg", "Pace"]:
    if base in features_all.columns:
        roll_col = f"{base}_g_roll5"
        if roll_col not in features_all.columns:
            features_all[roll_col] = (
                features_all.sort_values(["TEAM_ABBREVIATION","GAME_DATE"])
                            .groupby("TEAM_ABBREVIATION")[base]
                            .rolling(5, min_periods=1)
                            .mean()
                            .reset_index(level=0, drop=True)
            )
            print(f"✅ Created {roll_col}")
        context_bases.append(roll_col)

# If none of those exist, use generic team performance metrics instead
if not context_bases:
    print("⚠️ No ORtg/DRtg/Pace in data — using PTS_roll5/REB_roll5/AST_roll5 as proxies.")
    proxy_cols = [c for c in ["PTS_roll5","REB_roll5","AST_roll5"] if c in features_all.columns]
    for c in proxy_cols:
        features_all[f"team_{c}"] = (
            features_all.sort_values(["TEAM_ABBREVIATION","GAME_DATE"])
                        .groupby("TEAM_ABBREVIATION")[c]
                        .transform("mean")
        )
    context_bases = [f"team_{c}" for c in proxy_cols]

# 2️⃣ Build team-level table
agg_dict = {
    "team_pts": ("PTS","sum"),
    "team_pts_next": ("PTS_next","sum"),
}
for c in context_bases:
    agg_dict[c.replace("_g_roll5","").replace("team_","")] = (c,"mean")

team_games = (
    features_all.groupby(["GAME_DATE","TEAM_ABBREVIATION"], as_index=False)
    .agg(**agg_dict)
)

# 3️⃣ Build opponent-matched pairs (same date)
opp = team_games.rename(columns={
    "TEAM_ABBREVIATION": "OPPONENT_ABBREVIATION",
    "team_pts": "opp_pts",
    "team_pts_next": "opp_pts_next",
    **{k: f"opp_{k}" for k in [x for x in team_games.columns if x not in ["GAME_DATE","TEAM_ABBREVIATION","team_pts","team_pts_next"]]}
})
team_matchups = team_games.merge(opp, on=["GAME_DATE"], how="inner")

# 4️⃣ Train baseline Ridge model
team_feature_cols = [c for c in team_matchups.columns if c.startswith(("or","dr","pace","team_PTS","opp_")) and c != "team_pts_next"]

tm = team_matchups.dropna(subset=team_feature_cols + ["team_pts_next"]).copy()

if not tm.empty:
    X_tm = tm[team_feature_cols]
    y_tm = tm["team_pts_next"]
    ridge = Ridge(alpha=5.0).fit(X_tm, y_tm)
    print(f"✅ Team PTS baseline R²: {ridge.score(X_tm, y_tm):.3f}  |  n={len(X_tm):,}")
else:
    print("⚠️ Not enough data to train team baseline model.")


⚠️ No ORtg/DRtg/Pace in data — using PTS_roll5/REB_roll5/AST_roll5 as proxies.
✅ Team PTS baseline R²: 0.197  |  n=87,504


In [13]:
# pip install selenium webdriver-manager bs4 pandas lxml

import os, re, time, pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ---------------- helpers ----------------

def _clean_list(xs):
    return [re.sub(r"\s+\(.*?\)\s*$", "", x) for x in xs]

def _try_click_consent(driver, timeout=6):
    XPATHS = [
        "//button[contains(.,'Accept')]",
        "//button[contains(.,'I Agree')]",
        "//button[contains(.,'Agree')]",
        "//button[contains(.,'Αποδοχή')]",
        "//button[contains(.,'Συμφωνώ')]",
    ]
    end = time.time() + timeout
    for xp in XPATHS:
        try:
            btn = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, xp)))
            btn.click()
            return True
        except Exception:
            if time.time() > end: break
    return False

def _progress_scroll(driver, steps=10, pause=0.8):
    h = driver.execute_script("return document.body.scrollHeight || document.documentElement.scrollHeight;")
    for i in range(1, steps + 1):
        y = int(h * i / steps)
        driver.execute_script(f"window.scrollTo(0, {y});")
        time.sleep(pause)

def _extract_team(side):
    team_el = side.select_one(".lineup__abbr, .lineup__team-name, .lineup__name")
    if team_el:
        return team_el.get_text(strip=True)
    logo = side.select_one("img[alt]")
    return (logo.get("alt") or "").strip() if logo else ""

def _extract_status(side):
    status_el = side.select_one(".lineup__status")
    txt = (status_el.get_text(" ", strip=True) if status_el else "").upper()
    if "CONFIRM" in txt:  return "CONFIRMED"
    if "EXPECT" in txt or "PROBABLE" in txt: return "EXPECTED"
    return "UNKNOWN"

def _extract_starters(side):
    # Try several variants for starters content
    containers = side.select(".lineup__list--starters, .lineup__list, .lineup__players")
    if not containers:
        containers = [side]

    names = []
    for blk in containers:
        for a in blk.select("a.lineup__player-link, .lineup__player a"):
            t = a.get_text(" ", strip=True)
            if t: names.append(t)
        if not names:
            for row in blk.select(".lineup__player"):
                t = row.get_text(" ", strip=True)
                if re.match(r"^(PG|SG|SF|PF|C)\b", t): names.append(t)
        if not names:
            for li in blk.select("li"):
                t = li.get_text(" ", strip=True)
                if re.match(r"^(PG|SG|SF|PF|C)\b", t): names.append(t)

    if not names:
        txt = side.get_text("\n", strip=True)
        names = re.findall(r"(?:^|\n)(?:PG|SG|SF|PF|C)\s+[^\n]+", txt)

    return _clean_list(names)[:5]

# ---------------- main ----------------

def fetch_rotowire_lineups_selenium(date: str | None = None,
                                    wait_sec: float = 14.0,
                                    headless: bool = False) -> pd.DataFrame:
    """
    Render Rotowire lineups & parse BOTH sides per game (global side selectors).
    Returns:
      game_time, team, side (AWAY/HOME), lineup_status, starters,
      starter_1..starter_5, lineup_confirmed (0/1)
    """
    base = "https://www.rotowire.com/basketball/nba-lineups.php"
    url = base if not date else f"{base}?date={date}"

    opts = Options()
    if headless: opts.add_argument("--headless=new")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--window-size=1400,1000")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option("useAutomationExtension", False)
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_argument("--lang=en-US,en;q=0.9")
    opts.add_argument(
        "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    )

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    driver.get(url)

    _try_click_consent(driver, timeout=6)
    time.sleep(1.2)
    try:
        WebDriverWait(driver, int(wait_sec)).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".lineup, .lineup.is-nba"))
        )
    except Exception:
        pass

    _progress_scroll(driver, steps=10, pause=0.8)
    time.sleep(1.0)

    # quick diagnostics
    blocks = driver.find_elements(By.CSS_SELECTOR, ".lineup.is-nba, .lineup")
    players = driver.find_elements(By.CSS_SELECTOR, ".lineup__player, a.lineup__player-link")
    print(f"diagnostics: lineup blocks={len(blocks)}, player nodes={len(players)}")

    html = driver.page_source
    os.makedirs("_rotowire_debug", exist_ok=True)
    with open("_rotowire_debug/last_lineups.html", "w", encoding="utf-8") as f:
        f.write(html)
    try:
        driver.save_screenshot("_rotowire_debug/last_lineups.png")
    except Exception:
        pass
    driver.quit()

    # -------- parse globally by side classes ----------
    soup = BeautifulSoup(html, "lxml")

    # game time map: find each game container time
    game_time_map = {}
    for gi, g in enumerate(soup.select(".lineup__main, .lineup.is-nba, .lineup")):
        t = g.select_one(".lineup__time, .game-time")
        game_time_map[id(g)] = t.get_text(strip=True) if t else ""

    # Select **visit/away** & **home** side boxes explicitly
    visit_sel = (
        '[class*="lineup__box"][class*="is-visit"], '
        '[class*="lineup__team"][class*="is-visit"], '
        '[class*="lineup__side"][class*="is-visit"], '
        '[class*="visit"]'
    )
    home_sel = (
        '[class*="lineup__box"][class*="is-home"], '
        '[class*="lineup__team"][class*="is-home"], '
        '[class*="lineup__side"][class*="is-home"], '
        '[class*="home"]'
    )

    visit_boxes = soup.select(visit_sel)
    home_boxes  = soup.select(home_sel)

    rows = []

    def add_rows(boxes, side_label):
        for box in boxes:
            # nearest parent game container for time
            parent = box.find_parent(lambda tag: tag.has_attr("class") and any(
                c in {"lineup__main","lineup","lineup is-nba"} for c in tag.get("class", [])
            ))
            game_time = game_time_map.get(id(parent), "") if parent else ""
            team = _extract_team(box)
            starters = _extract_starters(box)
            status = _extract_status(box)
            if starters or team:
                rows.append({
                    "game_time": game_time,
                    "team": team,
                    "side": side_label,
                    "lineup_status": status,
                    "starters": starters,
                    "starter_1": starters[0] if len(starters)>0 else None,
                    "starter_2": starters[1] if len(starters)>1 else None,
                    "starter_3": starters[2] if len(starters)>2 else None,
                    "starter_4": starters[3] if len(starters)>3 else None,
                    "starter_5": starters[4] if len(starters)>4 else None,
                    "lineup_confirmed": int(status == "CONFIRMED"),
                })

    add_rows(visit_boxes, "AWAY")
    add_rows(home_boxes,  "HOME")

    df = pd.DataFrame(rows)

    if not df.empty:
        df = df.drop_duplicates(
            subset=["game_time","team","side","starter_1","starter_2","starter_3","starter_4","starter_5"]
        )
        all_na = df[["starter_1","starter_2","starter_3","starter_4","starter_5"]].isna().all(axis=1)
        df = df[~all_na].reset_index(drop=True)
    else:
        print("⚠️ Parsed zero rows. Check _rotowire_debug/last_lineups.html & .png")

    return df

# ---------- run it ----------
df_lineups = fetch_rotowire_lineups_selenium(wait_sec=14.0, headless=False)
print("✅ Shape:", df_lineups.shape)
print(df_lineups.sort_values(["game_time","side"]).head(12).to_string(index=False))



diagnostics: lineup blocks=12, player nodes=146
✅ Shape: (18, 11)
game_time team side lineup_status                                                                starters     starter_1     starter_2     starter_3        starter_4     starter_5  lineup_confirmed
               AWAY      EXPECTED    [Ryan Rollins, AJ Green, Gary Trent, G. Antetokounmpo, Myles Turner]  Ryan Rollins      AJ Green    Gary Trent G. Antetokounmpo  Myles Turner                 0
               AWAY      EXPECTED  [D. DiVincenzo, Mike Conley, J. McDaniels, Julius Randle, Rudy Gobert] D. DiVincenzo   Mike Conley  J. McDaniels    Julius Randle   Rudy Gobert                 0
               AWAY      EXPECTED   [K. George, S. Mykhailiuk, L. Markkanen, K. Filipowski, Jusuf Nurkic]     K. George S. Mykhailiuk  L. Markkanen    K. Filipowski  Jusuf Nurkic                 0
               AWAY      EXPECTED         [CJ McCollum, K. George, K. Middleton, B. Coulibaly, Alex Sarr]   CJ McCollum     K. George  K. Middleto

In [14]:
# pip install bs4 lxml pandas
import re, os, pandas as pd
from bs4 import BeautifulSoup

def _txt(x):
    return re.sub(r"\s+", " ", x.get_text(" ", strip=True)) if x else ""

def _clean_player(n):
    if not n: return n
    n = re.sub(r"\s+\(.*?\)\s*$", "", n).strip()
    n = re.sub(r"^(PG|SG|SF|PF|C)\s+", "", n, flags=re.I)
    return n

def _get_mnp_from_ul(ul):
    """Extract 'May Not Play' entries from a team UL."""
    mnp = []
    # Strategy 1: find the title li inside this UL, then collect following player lis until next title
    title = ul.find("li", class_=lambda c: c and "lineup__title" in c and re.search(r"may\s+not\s+play", _txt(ul.find("li", class_=c)) if ul.find("li", class_=c) else "", re.I))
    if title:
        for li in title.find_all_next("li"):
            # stop if next section title
            if "lineup__title" in (li.get("class") or []):
                break
            if "lineup__player" in (li.get("class") or []):
                a = li.select_one("a")
                tag = li.select_one(".lineup__inj")
                nm = _txt(a) if a else ""
                if nm:
                    mnp.append(f"{nm} ({_txt(tag)})" if tag else nm)
        # normalize
        return [_clean_player(x) for x in mnp if x and x.lower() != "none"]

    # Strategy 2: common MNP containers inside UL
    for li in ul.select(".lineup__notplay li, .lineup__status--out, .lineup__inj-list li"):
        nm = _txt(li)
        if nm: mnp.append(_clean_player(nm))
    return [x for x in mnp if x and x.lower() != "none"]

def _extract_starters_from_ul(ul):
    """Try multiple ways to get five starters out of a team UL."""
    names = []
    # Most reliable: 100% rows
    for li in ul.select("li.lineup__player.is-pct-play-100 a"):
        nm = _txt(li)
        if nm: names.append(nm)
    # Fallback: any lineup__player anchors in first list group
    if len(names) < 5:
        for li in ul.select("li.lineup__player a"):
            nm = _txt(li)
            if nm: names.append(nm)
            if len(names) >= 5: break
    # Final cleanup + trim
    names = [_clean_player(n) for n in names]
    return names[:5]

def _lineup_status(ul):
    st = _txt(ul.select_one(".lineup__status"))
    stU = st.upper()
    if "CONFIRM" in stU: return "CONFIRMED"
    if "EXPECT" in stU or "PROBABLE" in stU: return "EXPECTED"
    return "UNKNOWN"

def parse_rotowire_lineups_flexible(html_path: str) -> pd.DataFrame:
    with open(html_path, "r", encoding="utf-8", errors="ignore") as f:
        html = f.read()
    soup = BeautifulSoup(html, "lxml")

    # --- Diagnostics to understand the DOM you have ---
    diag = {
        "lineup__teams": len(soup.select("div.lineup__teams")),
        "ul.lineup__list": len(soup.select("ul.lineup__list")),
        "ul.is-visit": len(soup.select("ul.lineup__list.is-visit")),
        "ul.is-home": len(soup.select("ul.lineup__list.is-home")),
        "see-proj-minutes buttons": len(soup.select("button.see-proj-minutes")),
        "header abbr": len(soup.select(".lineup__hdr .lineup__abbr")),
        "header team": len(soup.select(".lineup__hdr .lineup__team")),
        "player anchors": len(soup.select("a.lineup__player-link, .lineup__player a")),
        "MNP titles": len(soup.find_all(string=re.compile(r"^\s*may\s+not\s+play\s*$", re.I))),
    }
    print("DOM diagnostics:", diag)

    rows = []

    # ========== STRATEGY A: by matchup blocks ==========
    for teams_div in soup.select("div.lineup__teams"):
        # game time near this block (looks upward for a sibling header)
        time_el = teams_div.find_previous("div", class_="lineup__time")
        game_time = _txt(time_el)

        # find both team ULs inside this matchup
        uls = teams_div.select("ul.lineup__list")
        if len(uls) < 1:
            continue

        # Try to pair AWAY then HOME by class flags; else preserve order
        away_ul = None
        home_ul = None
        for ul in uls:
            classes = " ".join(ul.get("class", [])).lower()
            if "is-visit" in classes or "visit" in classes or "away" in classes:
                away_ul = ul
            if "is-home" in classes or "home" in classes:
                home_ul = home_ul or ul  # keep the first

        if away_ul is None and home_ul is None and len(uls) >= 2:
            away_ul, home_ul = uls[0], uls[1]
        elif away_ul is None and len(uls) >= 1:
            away_ul = uls[0]
        elif home_ul is None and len(uls) >= 2:
            # pick the other UL as home
            home_ul = next((u for u in uls if u is not away_ul), None)

        pairs = [("AWAY", away_ul), ("HOME", home_ul)]
        # Extract team code (prefer button data-team; else header abbrs in the same matchup)
        header_abbrs = [ _txt(el) for el in teams_div.select(".lineup__abbr") if _txt(el) ]
        # If header not inside teams_div, try its parent block
        if not header_abbrs:
            parent_main = teams_div.find_parent(["div","section"])
            if parent_main:
                header_abbrs = [ _txt(el) for el in parent_main.select(".lineup__abbr") if _txt(el) ]

        for idx, (side, ul) in enumerate(pairs):
            if not ul: continue
            btn = ul.select_one("button.see-proj-minutes")
            team = btn["data-team"].strip().upper() if btn and btn.has_attr("data-team") else None
            if not team and header_abbrs and idx < len(header_abbrs):
                team = header_abbrs[idx].upper()

            starters = _extract_starters_from_ul(ul)
            mnp = _get_mnp_from_ul(ul)
            status = _lineup_status(ul)

            # Only add if we have at least a team or any player info
            if team or starters or mnp:
                rows.append({
                    "game_time": game_time,
                    "team": team,
                    "side": side,
                    "lineup_status": status,
                    "starters": starters,
                    "may_not_play": mnp,
                    "may_not_play_count": len(mnp),
                    "lineup_confirmed": int(status == "CONFIRMED"),
                })

    # ========== STRATEGY B: fall back to any lineup ULs globally ==========
    if not rows:
        print("Fallback B: scanning all ul.lineup__list globally...")
        for ul in soup.select("ul.lineup__list"):
            # Guess side by class or position among siblings
            side = "AWAY" if "is-visit" in (ul.get("class") or []) else ("HOME" if "is-home" in (ul.get("class") or []) else None)
            # Team from button
            btn = ul.select_one("button.see-proj-minutes")
            team = btn["data-team"].strip().upper() if btn and btn.has_attr("data-team") else None
            starters = _extract_starters_from_ul(ul)
            mnp = _get_mnp_from_ul(ul)
            status = _lineup_status(ul)

            if side and (team or starters or mnp):
                rows.append({
                    "game_time": "",  # unknown at this scope
                    "team": team,
                    "side": side,
                    "lineup_status": status,
                    "starters": starters,
                    "may_not_play": mnp,
                    "may_not_play_count": len(mnp),
                    "lineup_confirmed": int(status == "CONFIRMED"),
                })

    # ========== STRATEGY C: header-driven pairing (very defensive) ==========
    if not rows:
        print("Fallback C: pairing by header labels and nearest lists...")
        for block in soup.select(".lineup, .lineup__main"):
            hdr = block.select(".lineup__hdr .lineup__abbr, .lineup__hdr .lineup__team")
            labels = [ _txt(x) for x in hdr if _txt(x) ]
            if len(labels) < 2:
                continue
            away_label, home_label = labels[:2]
            lists = block.select("ul.lineup__list")
            if len(lists) < 2:
                continue
            for side, lab, ul in [("AWAY", away_label, lists[0]), ("HOME", home_label, lists[1])]:
                starters = _extract_starters_from_ul(ul)
                mnp = _get_mnp_from_ul(ul)
                status = _lineup_status(ul)
                rows.append({
                    "game_time": _txt(block.select_one(".lineup__time, .game-time")),
                    "team": lab.upper(),
                    "side": side,
                    "lineup_status": status,
                    "starters": starters,
                    "may_not_play": mnp,
                    "may_not_play_count": len(mnp),
                    "lineup_confirmed": int(status == "CONFIRMED"),
                })

    df = pd.DataFrame(rows)
    # Expand starters to columns for easier merging
    for i in range(5):
        col = f"starter_{i+1}"
        df[col] = df["starters"].apply(lambda xs: xs[i] if isinstance(xs, list) and len(xs) > i else None)

    print(f"→ Parsed rows: {len(df)}")
    return df

# ---- RUN IT (point to your saved file) ----
HTML_PATH = "_rotowire_debug/last_lineups.html"  # change if needed
if not os.path.exists(HTML_PATH):
    # if you uploaded as 'last_lineups.html' in current directory
    if os.path.exists("last_lineups.html"):
        HTML_PATH = "last_lineups.html"

df_lineups = parse_rotowire_lineups_flexible(HTML_PATH)

# Safe display
if df_lineups.empty:
    print("\n⚠️ Still empty. Please share the values printed in 'DOM diagnostics' (above).")
else:
    cols = ["game_time","team","side","lineup_status","may_not_play_count",
            "starter_1","starter_2","starter_3","starter_4","starter_5"]
    print("\n✅ Preview:")
    print(df_lineups[cols].sort_values(["game_time","side","team"], na_position="last").to_string(index=False))


DOM diagnostics: {'lineup__teams': 9, 'ul.lineup__list': 18, 'ul.is-visit': 9, 'ul.is-home': 9, 'see-proj-minutes buttons': 18, 'header abbr': 0, 'header team': 0, 'player anchors': 146, 'MNP titles': 18}
Fallback B: scanning all ul.lineup__list globally...
→ Parsed rows: 18

✅ Preview:
game_time team side lineup_status  may_not_play_count     starter_1        starter_2     starter_3        starter_4     starter_5
           DAL AWAY      EXPECTED                   9  Cooper Flagg     Max Christie Klay Thompson    P. Washington    D. Gafford
           DET AWAY      EXPECTED                   8 C. Cunningham      D. Robinson   A. Thompson      Jalen Duren C. Cunningham
           LAL AWAY      EXPECTED                  12  Marcus Smart     Jake LaRavia Rui Hachimura    J. Vanderbilt  Marcus Smart
           MIA AWAY      EXPECTED                   9   D. Mitchell       A. Wiggins   Bam Adebayo      Kel'el Ware   D. Mitchell
           MIL AWAY      EXPECTED                   6  Ryan Ro

In [ ]:
# pip install bs4 lxml pandas
import os, re, pandas as pd
from bs4 import BeautifulSoup

HTML_PATH = "_rotowire_debug/last_lineups.html" if os.path.exists("_rotowire_debug/last_lineups.html") else "last_lineups.html"

LIKELIHOOD_MAP = {
    "is-pct-play-100": 100, "is-pct-play-90": 90, "is-pct-play-75": 75,
    "is-pct-play-60": 60, "is-pct-play-50": 50, "is-pct-play-40": 40,
    "is-pct-play-25": 25, "is-pct-play-10": 10, "is-pct-play-0": 0
}

def _txt(node): return re.sub(r"\s+", " ", node.get_text(" ", strip=True)) if node else ""
def _likelihood(classes): 
    for c in classes: 
        if c in LIKELIHOOD_MAP: 
            return LIKELIHOOD_MAP[c]
    return None

def parse_rotowire_mnp_final(html_path: str) -> pd.DataFrame:
    with open(html_path, "r", encoding="utf-8", errors="ignore") as f:
        soup = BeautifulSoup(f.read(), "lxml")

    rows = []
    games = soup.select("div.lineup.is-nba[data-lnum]")
    print(f"Found {len(games)} games in HTML.")

    for game in games:
        game_time = _txt(game.select_one(".lineup__time"))
        team_blocks = game.select(".lineup__team")
        teams = []
        for tb in team_blocks:
            abbr = _txt(tb.select_one(".lineup__abbr"))
            side = "AWAY" if "is-visit" in tb.get("class", []) else "HOME" if "is-home" in tb.get("class", []) else None
            teams.append((abbr, side))

        ul_lists = game.select("ul.lineup__list")
        for idx, ul in enumerate(ul_lists):
            if idx >= len(teams):  # mismatch safety
                continue
            team, side = teams[idx]
            mnp_title = ul.find("li", class_="lineup__title", string=lambda s: s and "MAY NOT PLAY" in s.upper())
            if not mnp_title:
                continue

            for li in mnp_title.find_next_siblings("li"):
                classes = li.get("class") or []
                if "lineup__title" in classes:
                    break
                if "lineup__player" not in classes:
                    continue

                pos = _txt(li.select_one(".lineup__pos"))
                a = li.select_one("a")
                player = _txt(a)
                if not player:
                    continue

                status = _txt(li.select_one(".lineup__inj"))
                title_text = (li.get("title") or "").strip()
                likelihood_pct = _likelihood(classes)

                rows.append({
                    "game_time": game_time,
                    "team": team,
                    "side": side,
                    "position": pos,
                    "player": player,
                    "status": status,
                    "title_text": title_text,
                    "likelihood_pct": likelihood_pct
                })

    df = pd.DataFrame(rows)
    if df.empty:
        print("⚠️ No 'May Not Play' players found. Check if Rotowire changed markup.")
    else:
        df = df.sort_values(["game_time","side","team","player"]).reset_index(drop=True)
        print(f"✅ Parsed {len(df)} 'May Not Play' players across {df['team'].nunique()} teams.")
    return df


# ---- RUN ----
mnp_df = parse_rotowire_mnp_final(HTML_PATH)
if not mnp_df.empty:
    print(mnp_df.head(30).to_string(index=False))
    mnp_df.to_csv(os.path.join(FOLDER_EVALS, f"value_bets_top100_{TODAY}_EVAL.csv"), index=False)
    print("\nSaved: may_not_play_players.csv")


Found 9 games in HTML.
✅ Parsed 56 'May Not Play' players across 18 teams.
  game_time team side position        player status            title_text  likelihood_pct
10:00 PM ET  LAL AWAY        G     A. Reaves    Out Very Unlikely To Play               0
10:00 PM ET  LAL AWAY        F     A. Thiero    Out Very Unlikely To Play               0
10:00 PM ET  LAL AWAY        C      D. Ayton   Ques       Toss Up To Play              50
10:00 PM ET  LAL AWAY        G    G. Vincent    Out Very Unlikely To Play               0
10:00 PM ET  LAL AWAY        G     L. Doncic    Out Very Unlikely To Play               0
10:00 PM ET  LAL AWAY        F      L. James    Out Very Unlikely To Play               0
10:00 PM ET  LAL AWAY        C     M. Kleber   Ques       Toss Up To Play              50
10:00 PM ET  POR HOME        G     B. Wesley    Out Very Unlikely To Play               0
10:00 PM ET  POR HOME        G    D. Lillard    OFS   Very Likely To Play               0
10:00 PM ET  POR HOME    

In [16]:
import pandas as pd
import numpy as np
from datetime import datetime

def american_to_prob(odds):
    if pd.isna(odds): return np.nan
    o = float(odds)
    return 100.0/(o+100.0) if o>0 else (-o)/(-o+100.0)

def devig_pair(p_over, p_under):
    if pd.isna(p_over) or pd.isna(p_under): return (np.nan, np.nan)
    s = p_over + p_under
    if s <= 0: return (np.nan, np.nan)
    return (p_over/s, p_under/s)

def kelly_fraction(p, american_odds, cap=0.25):
    if pd.isna(p) or pd.isna(american_odds): return 0.0
    o = float(american_odds)
    b = o/100.0 if o>0 else 100.0/(-o)
    f = (p*(b+1)-1)/b
    return float(max(0.0, min(f, cap)))

def ev_flat_over(p, american_odds):
    if pd.isna(p) or pd.isna(american_odds): return np.nan
    o = float(american_odds)
    win = o/100.0 if o>0 else 100.0/(-o)
    lose = 1.0
    return p*win - (1-p)*lose

# Normal CDF helper (if SciPy available) to turn mean/sd into p_over
try:
    from scipy.stats import norm
    def p_over_from_normal(mu, sd, line):
        if pd.isna(mu) or pd.isna(sd) or pd.isna(line) or sd <= 0: return np.nan
        return 1.0 - norm.cdf((line - mu)/sd)
except Exception:
    def p_over_from_normal(mu, sd, line): return np.nan

def build_value_bets_excel(
    df_projections, df_odds, outfile_path=None,
    join_keys=("player","team","opponent","market","line","book","game_date"),
    cap_kelly=0.25
):
    def _norm(x): return None if pd.isna(x) else str(x).strip()
    proj, odds = df_projections.copy(), df_odds.copy()
    for k in join_keys:
        if k in proj: proj[k] = proj[k].map(_norm)
        if k in odds: odds[k] = odds[k].map(_norm)

    merged = proj.merge(odds, on=list(join_keys), how="inner", suffixes=("", "_odds"))

    if "p_over_model" not in merged.columns or merged["p_over_model"].isna().all():
        merged["p_over_model"] = merged.apply(
            lambda r: p_over_from_normal(r.get("projection_mean"), r.get("projection_sd"), r.get("line")), axis=1
        )

    merged["p_over_imp"]  = merged["over_odds"].map(american_to_prob)
    merged["p_under_imp"] = merged["under_odds"].map(american_to_prob)
    merged[["p_over_fair","p_under_fair"]] = merged.apply(
        lambda r: pd.Series(devig_pair(r["p_over_imp"], r["p_under_imp"])), axis=1
    )

    merged["edge_over"]       = merged["p_over_model"] - merged["p_over_fair"]
    merged["kelly_frac_over"] = merged.apply(lambda r: kelly_fraction(r["p_over_model"], r["over_odds"], cap=cap_kelly), axis=1)
    merged["EV_over_1u"]      = merged.apply(lambda r: ev_flat_over(r["p_over_model"], r["over_odds"]), axis=1)
    merged["asof_date"]       = merged.get("asof_date") if "asof_date" in merged else datetime.utcnow().strftime("%Y-%m-%d")

    preferred = [
        "asof_date","game_date","book","player","team","opponent","market","line","lineup_status",
        "over_odds","under_odds","p_over_imp","p_under_imp","p_over_fair","p_under_fair","p_over_model",
        "edge_over","kelly_frac_over","EV_over_1u",
        "projected_minutes","projection_mean","projection_sd","start_prob",
        "opponent_allowance_idx","team_orating","opp_drating",
    ]
    cols = [c for c in preferred if c in merged.columns] + [c for c in merged.columns if c not in preferred]
    bets = merged[cols].sort_values(["edge_over","EV_over_1u"], ascending=False).reset_index(drop=True)

    summary = pd.DataFrame({
        "n_bets":[len(bets)],
        "avg_edge_pp":[bets["edge_over"].mean()*100.0 if len(bets) else np.nan],
        "avg_kelly_pct":[bets["kelly_frac_over"].mean()*100.0 if len(bets) else np.nan],
        "avg_ev_1u":[bets["EV_over_1u"].mean() if len(bets) else np.nan],
    })
    by_market = bets.groupby("market", dropna=False).agg(
        n=("player","count"),
        avg_edge_pp=("edge_over", lambda x: 100.0*x.mean()),
        avg_kelly_pct=("kelly_frac_over", lambda x: 100.0*x.mean()),
        avg_ev_1u=("EV_over_1u","mean")
    ).reset_index()
    by_book = bets.groupby("book", dropna=False).agg(
        n=("player","count"),
        avg_edge_pp=("edge_over", lambda x: 100.0*x.mean()),
        avg_ev_1u=("EV_over_1u","mean")
    ).reset_index()

    if outfile_path is None:
        outfile_path = f"nba_value_bets_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}.xlsx"
    with pd.ExcelWriter(outfile_path, engine="openpyxl") as w:
        bets.to_excel(w, sheet_name="Bets", index=False)
        summary.to_excel(w, sheet_name="Summary", index=False, startrow=0)
        by_market.to_excel(w, sheet_name="Summary", index=False, startrow=5)
        by_book.to_excel(w, sheet_name="Summary", index=False, startrow=5+len(by_market)+3)

        dd = pd.DataFrame([
            ("asof_date","UTC run date"), ("game_date","Game date"),
            ("player","Player"), ("team","Team abbr"), ("opponent","Opponent abbr"),
            ("market","PTS/REB/AST/3PM/PRA etc."), ("line","Book line"), ("book","Sportsbook id"),
            ("lineup_status","EXPECTED/CONFIRMED/UNKNOWN"),
            ("over_odds","American odds Over"), ("under_odds","American odds Under"),
            ("p_over_imp","Implied prob Over (pre-vig)"), ("p_under_imp","Implied prob Under (pre-vig)"),
            ("p_over_fair","De-vigged prob Over"), ("p_under_fair","De-vigged prob Under"),
            ("p_over_model","Model prob Over"), ("edge_over","p_model − p_fair"),
            ("kelly_frac_over","Kelly fraction (cap)"), ("EV_over_1u","EV if staking 1u"),
            ("projected_minutes","Projected minutes"), ("projection_mean","Projected mean"),
            ("projection_sd","Projected stdev"), ("start_prob","Start probability"),
            ("opponent_allowance_idx","Opponent allowance index"),
            ("team_orating","Team ORtg"), ("opp_drating","Opponent DRtg"),
        ], columns=["column","description"])
        dd.to_excel(w, sheet_name="Data_Dictionary", index=False)

    return bets, outfile_path


In [17]:
# === 16: raw wide odds + resilient numeric parsing ===
import re, json, pandas as pd
from datetime import datetime

def _first_numeric_float(x):
    """Return the first decimal number in x (e.g., '23.5, 24.5' -> 23.5)."""
    if x is None: return None
    s = str(x)
    m = re.search(r"[-+]?\d+(?:\.\d+)?", s)
    return float(m.group()) if m else None

def _first_numeric_int(x):
    """Return the first integer in x (e.g., '+110, +105' -> 110)."""
    if x is None: return None
    s = str(x)
    m = re.search(r"[-+]?\d+", s)
    return int(m.group()) if m else None

# override the helpers used by 16d converter (if defined)
def _to_float_or_none(x):  # noqa: F811
    return _first_numeric_float(x)

def _to_int_or_none(x):    # noqa: F811
    return _first_numeric_int(x)

def get_player_props_odds_wide_raw(self, book="mgm"):
    """
    Returns the raw 'wide' odds table from Rotowire (no grouping, no aggregation).
    Contains columns like mgm_pts, mgm_ptsOver, mgm_ptsUnder, etc.
    """
    url = f"https://www.rotowire.com/betting/nba/player-props.php?book={book}"
    r = self.session.get(url, headers=self.headers, timeout=30)
    r.raise_for_status()
    matches = re.findall(r'data:\s*(\[\{.*?\}\])', r.text, flags=re.DOTALL)
    frames = []
    for blob in matches:
        try:
            frames.append(pd.DataFrame(json.loads(blob)))
        except Exception:
            pass
    if not frames:
        return pd.DataFrame()
    # concat all blocks without grouping to preserve raw book columns
    wide_raw = pd.concat(frames, ignore_index=True)
    return wide_raw

# attach to your scraper class
NBAOddsAndLineupsScraper.get_player_props_odds_wide_raw = get_player_props_odds_wide_raw


In [22]:
# === Cell 16 (Upgraded): projections for PTS/REB/AST using trained XGB models ===
import pandas as pd
import numpy as np
from datetime import datetime

# Detect model dictionary
if "models_mean" in globals() and models_mean:
    active_models = models_mean
    feature_dict = feature_bags
elif "models" in globals() and models:
    active_models = models
    feature_dict = feature_cols_by_stat
else:
    raise RuntimeError("❌ No trained models found. Run Cell 8 first to populate models_mean and feature_bags.")

MARKETS = ["PTS", "REB", "AST"]

# 1️⃣ Latest row per player = baseline for next game prediction
latest = (
    features_all.sort_values(["PLAYER_NAME", "GAME_DATE"])
    .groupby("PLAYER_NAME")
    .tail(1)
    .copy()
)

# 2️⃣ Estimate per-player variability (standard deviation of recent games)
def _player_sd_map(stat: str, n=10):
    def _sd(g):
        s = g[stat].tail(n)
        if s.notna().sum() >= 4:
            return float(s.std(ddof=1))
        return float(features_all[stat].std(ddof=1))
    return features_all.groupby("PLAYER_NAME").apply(_sd)

# 3️⃣ Common export keys
base_out = pd.DataFrame({
    "player": latest["PLAYER_NAME"],
    "team": latest["TEAM_ABBREVIATION"],
    "opponent": latest.get("OPPONENT_ABBREVIATION", "TBD"),
    "game_date": datetime.utcnow().strftime("%Y-%m-%d"),
    "projected_minutes": latest.get("MIN_roll5", 30).fillna(30).clip(lower=10, upper=40).values,
    "start_prob": 0.90,
    "lineup_status": "EXPECTED"
})

proj_frames = {}

# 4️⃣ Generate predictions per market
for stat in MARKETS:
    if stat not in active_models:
        print(f"⚠️ Skipping {stat}: model not found.")
        continue

    feat_cols = feature_dict.get(stat, [])
    if not feat_cols:
        print(f"⚠️ Skipping {stat}: no feature list recorded.")
        continue

    X_pred = latest[feat_cols].fillna(0)
    pred_mean = active_models[stat].predict(X_pred)

    # Per-player SD (fallback if not enough data)
    sd_map = _player_sd_map(stat)
    pred_sd = latest["PLAYER_NAME"].map(sd_map)
    sd_fallback = np.maximum(np.abs(pred_mean) * 0.15, 1.0)
    pred_sd = np.where(np.isnan(pred_sd), sd_fallback, pred_sd)

    dfp = base_out.copy()
    dfp["projection_mean"] = pred_mean
    dfp["projection_sd"] = pred_sd
    dfp["market"] = stat

    proj_frames[stat] = dfp[
        ["player","team","opponent","game_date","market",
         "projection_mean","projection_sd","projected_minutes",
         "start_prob","lineup_status"]
    ].copy()

# 5️⃣ Combined projections
df_projections_pts = proj_frames.get("PTS", pd.DataFrame())
df_projections_reb = proj_frames.get("REB", pd.DataFrame())
df_projections_ast = proj_frames.get("AST", pd.DataFrame())
df_projections_all = (
    pd.concat(list(proj_frames.values()), ignore_index=True)
    if proj_frames else pd.DataFrame()
)

print("✅ Projection rows by market:",
      {k: len(v) for k, v in proj_frames.items()})


✅ Projection rows by market: {'PTS': 693, 'REB': 693, 'AST': 693}


In [23]:
# --- Helper: turn wide props (per-book columns) into a long, tidy table ---
import re
import numpy as np
import pandas as pd

def odds_wide_to_long_from_columns(
    wide_df: pd.DataFrame,
    *,
    books: tuple[str, ...] = ("mgm","draftkings","fanduel","caesars","betrivers","espnbet","hardrock"),
    markets: tuple[str, ...] = ("PTS","REB","AST"),
    player_cols=("name","player","PLAYER_NAME"),
    team_cols=("team","TEAM","team_name","TEAM_ABBREVIATION"),
    opp_cols=("opponent","opp","OPPONENT","OPPONENT_ABBREVIATION"),
    date_cols=("game_date","GAME_DATE","date")
) -> pd.DataFrame:
    """
    Convert a 'wide' props frame into a tidy long format:
    one row per (player, market, book), with numeric line and American odds.

    Expected column patterns (flexible by regex):
      <book>_<suffix>                 -> the line (e.g., mgm_pts, fanduel_ast)
      <book>_<suffix>_over_odds       -> over odds (American)
      <book>_<suffix>_under_odds      -> under odds (American)

    Suffixes recognized per market:
      PTS:  'pts','points'
      REB:  'reb','rebounds'
      AST:  'ast','assists'
    """
    df = wide_df.copy()

    # Identify reference columns
    def _first_col(cands):
        for c in cands:
            if c in df.columns: return c
        return None

    player_col = _first_col(player_cols)
    team_col   = _first_col(team_cols)
    opp_col    = _first_col(opp_cols)
    date_col   = _first_col(date_cols)

    # Fallbacks if totally missing
    if player_col is None:
        raise ValueError("Could not find a player name column in wide_df. "
                         f"Tried {player_cols}. Got columns: {list(df.columns)[:20]}...")

    # Normalize helpers
    def _num_float(x):
        if pd.isna(x): return np.nan
        m = re.search(r"[-+]?\d+(?:\.\d+)?", str(x))
        return float(m.group()) if m else np.nan

    def _num_int(x):
        if pd.isna(x): return np.nan
        m = re.search(r"[-+]?\d+", str(x))
        return int(m.group()) if m else np.nan

    # Market suffix map (flex)
    market_suffixes = {
        "PTS": ("pts","points"),
        "REB": ("reb","rebounds"),
        "AST": ("ast","assists"),
    }

    # Build long rows
    long_rows = []
    # Iterate rows once; pull columns per book/market dynamically
    for _, row in df.iterrows():
        base = {
            "player": row[player_col],
            "team": row[team_col] if team_col else np.nan,
            "opponent": row[opp_col] if opp_col else np.nan,
            "game_date": row[date_col] if date_col else np.nan,
        }
        for mkt in markets:
            suffixes = market_suffixes.get(mkt, ())
            for b in books:
                # Find the *line* column by trying allowed suffixes
                line_val = np.nan
                over_val = np.nan
                under_val = np.nan
                line_col_used = None

                for suf in suffixes:
                    # exact line column (most common)
                    c_line = f"{b}_{suf}"
                    if c_line in df.columns and pd.notna(row[c_line]):
                        line_val = row[c_line]
                        line_col_used = c_line
                        # odds columns (several sites use these names)
                        for over_name in (f"{b}_{suf}_over_odds", f"{b}_{suf}_o_odds", f"{b}_{suf}_over"):
                            if over_name in df.columns:
                                over_val = row[over_name]
                                break
                        for under_name in (f"{b}_{suf}_under_odds", f"{b}_{suf}_u_odds", f"{b}_{suf}_under"):
                            if under_name in df.columns:
                                under_val = row[under_name]
                                break
                        break  # found a suffix match

                # If not found, try a looser search (e.g., 'mgm_pts_line')
                if (isinstance(line_val, float) and np.isnan(line_val)) or line_col_used is None:
                    pat = re.compile(rf"^{re.escape(b)}_({ '|'.join(map(re.escape, suffixes)) })(_line)?$", re.I)
                    for c in df.columns:
                        if pat.match(str(c)) and pd.notna(row[c]):
                            line_val = row[c]
                            line_col_used = c
                            # odds columns with same base
                            base_prefix = re.sub(r"(_line)?$", "", c)
                            for over_name in (f"{base_prefix}_over_odds", f"{base_prefix}_o_odds", f"{base_prefix}_over"):
                                if over_name in df.columns:
                                    over_val = row[over_name]
                                    break
                            for under_name in (f"{base_prefix}_under_odds", f"{base_prefix}_u_odds", f"{base_prefix}_under"):
                                if under_name in df.columns:
                                    under_val = row[under_name]
                                    break
                            break

                # Only emit a row if we actually found a line
                if pd.notna(line_val):
                    long_rows.append({
                        **base,
                        "market": mkt,
                        "book": b,
                        "line": _num_float(line_val),
                        "over_odds": _num_int(over_val),
                        "under_odds": _num_int(under_val),
                    })

    out = pd.DataFrame(long_rows)

    # Clean up: drop obviously invalid lines
    if not out.empty:
        out = out[pd.notna(out["line"])]
        # remove zero/negative lines that can't be real for these markets (optional)
        out = out[out["line"] > 0]

        # De-duplicate best-effort (sometimes the page contains duplicates per book)
        out = (out.sort_values(["player","market","book","line"])
                  .drop_duplicates(subset=["player","market","book"], keep="last")
                  .reset_index(drop=True))

    return out


In [24]:
# === Cell: Define output folders for new XGBoost model ===
import os
from datetime import datetime

BASE_DIR = "xgb_outputs"
TODAY = datetime.utcnow().strftime("%Y%m%d")

# Folders for different result types
FOLDER_PROJECTIONS = os.path.join(BASE_DIR, "projections")
FOLDER_BETS        = os.path.join(BASE_DIR, "value_bets")
FOLDER_EVALS       = os.path.join(BASE_DIR, "evaluations")

for folder in [BASE_DIR, FOLDER_PROJECTIONS, FOLDER_BETS, FOLDER_EVALS]:
    os.makedirs(folder, exist_ok=True)

print(f"✅ Output directories ready under: {BASE_DIR}")


✅ Output directories ready under: xgb_outputs


In [25]:
# === Cell 17: wide_raw → long (PTS/REB/AST) → join → export ===
from datetime import datetime
import re, unicodedata
import numpy as np
import pandas as pd

scraper = NBAOddsAndLineupsScraper()

# 1) Raw wide odds (no grouping)
wide_raw = scraper.get_player_props_odds_wide_raw(book="mgm")
if wide_raw.empty:
    raise RuntimeError("Raw wide odds are empty. The page structure may have changed or was blocked.")

print("Total rows in raw wide:", len(wide_raw))

# 2) Detect books present
books_seen = sorted({
    m.group(1) for c in wide_raw.columns
    if (m := re.match(r"^(draftkings|fanduel|caesars|betrivers|espnbet|hardrock|mgm)_(.+)$", c))
})
print("Books present in columns:", books_seen)

def _col_exists_nonnull(df, col):
    return (col in df.columns) and df[col].notna().any()

# Which books have each market today?
market_suffix = {"PTS":"pts","REB":"reb","AST":"ast"}
books_by_market = {
    m: [b for b in books_seen if _col_exists_nonnull(wide_raw, f"{b}_{market_suffix[m]}")]
    for m in ["PTS","REB","AST"]
}
print("Books with lines:", {m: v for m, v in books_by_market.items() if v})

# 3) Convert wide → long for markets that actually have any lines
target_markets = tuple([m for m, bs in books_by_market.items() if bs])
if not target_markets:
    raise RuntimeError("No books have non-null PTS/REB/AST lines today.")

odds_long = odds_wide_to_long_from_columns(
    wide_raw,
    books=tuple(sorted({b for bs in books_by_market.values() for b in bs})),
    markets=target_markets
)
if odds_long.empty:
    raise RuntimeError("odds_long is empty after conversion. Verify your `odds_wide_to_long_from_columns` mapping.")

# Normalize obvious numerics
def _num_float(x):
    if pd.isna(x): return np.nan
    m = re.search(r"[-+]?\d+(?:\.\d+)?", str(x))
    return float(m.group()) if m else np.nan

def _num_int(x):
    if pd.isna(x): return np.nan
    m = re.search(r"[-+]?\d+", str(x))
    return int(m.group()) if m else np.nan

odds_long["line"] = odds_long["line"].apply(_num_float)
odds_long["over_odds"] = odds_long["over_odds"].apply(_num_int)
odds_long["under_odds"] = odds_long["under_odds"].apply(_num_int)

print("odds_long rows:", len(odds_long))
print("odds_long columns:", odds_long.columns.tolist())
print(odds_long.head(5))

# 4) Prepare projections union (must be created earlier, e.g., Cell 16e)
if "df_projections_all" not in globals() or df_projections_all.empty:
    raise RuntimeError("df_projections_all not found or empty (run Cell 16 that builds PTS/REB/AST projections).")

# Light name normalizer
def _norm_player(name: str) -> str:
    if not isinstance(name, str): return ""
    s = unicodedata.normalize("NFKD", name)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[.\-`'’]", "", s).strip().lower()
    s = re.sub(r"\s+", " ", s)
    return s

odds_long = odds_long.copy()
df_projections_all = df_projections_all.copy()

odds_long["player_key"] = odds_long["player"].map(_norm_player)
df_projections_all["player_key"] = df_projections_all["player"].map(_norm_player)

# Ensure projection SD exists per market (fallback to 15% of mean, min 1.0)
for m in ["PTS","REB","AST"]:
    mask = df_projections_all["market"].eq(m)
    if "projection_sd" not in df_projections_all.columns:
        df_projections_all["projection_sd"] = np.nan
    missing_sd = df_projections_all.loc[mask, "projection_sd"].isna() | (df_projections_all.loc[mask, "projection_sd"] <= 0)
    if missing_sd.any():
        df_projections_all.loc[mask, "projection_sd"] = (
            df_projections_all.loc[mask, "projection_mean"].abs() * 0.15
        ).clip(lower=1.0)

# 5) Split and join per market, then combine
joined_frames = []
for mkt in target_markets:
    odds_m = odds_long.loc[odds_long["market"].eq(mkt)].copy()
    proj_m = df_projections_all.loc[df_projections_all["market"].eq(mkt)].copy()
    if odds_m.empty or proj_m.empty:
        print(f"⚠️ Skipping {mkt}: odds or projections empty.")
        continue

    join_keys = ["player_key","market"]
    view_cols_odds = join_keys + ["player","team","opponent","line","book","game_date","over_odds","under_odds"]
    view_cols_odds = [c for c in view_cols_odds if c in odds_m.columns]

    view_cols_proj = join_keys + [
        "player","team","opponent","game_date","projection_mean","projection_sd",
        "projected_minutes","start_prob","lineup_status"
    ]
    view_cols_proj = [c for c in view_cols_proj if c in proj_m.columns]

    dfj = proj_m[view_cols_proj].merge(
        odds_m[view_cols_odds].rename(columns={"player":"player_odds","team":"team_odds","opponent":"opponent_odds","game_date":"game_date_odds"}),
        on=join_keys, how="inner", suffixes=("_proj","_odds")
    )

    if dfj.empty:
        print(f"⚠️ Join produced 0 rows for {mkt}. Check name variants.")
        continue

    # Resolve canonical columns
    def _pick_first(df_, names, default=np.nan):
        for n in names:
            if n in df_.columns:
                return df_[n]
        return default

    dfj = dfj.loc[:, ~dfj.columns.duplicated()].copy()
    dfj["player"]    = _pick_first(dfj, ["player_odds","player_proj","player"])
    dfj["team"]      = _pick_first(dfj, ["team_odds","team_proj","team"])
    dfj["opponent"]  = _pick_first(dfj, ["opponent_odds","opponent_proj","opponent"])
    dfj["game_date"] = _pick_first(dfj, ["game_date_odds","game_date_proj","game_date"])
    # line already numeric above, but if any slipped through:
    dfj["line"] = dfj["line"].apply(_num_float)

    # Compute model probability P(Over)
    if "p_over_from_normal" not in globals():
        from statistics import NormalDist
        def p_over_from_normal(mu, sd, line):
            if pd.isna(mu) or pd.isna(sd) or pd.isna(line) or float(sd) <= 0: return np.nan
            z = (float(line) - float(mu)) / float(sd)
            return 1.0 - NormalDist().cdf(z)

    dfj["p_over_model"] = dfj.apply(
        lambda r: p_over_from_normal(r.get("projection_mean"), r.get("projection_sd"), r.get("line")), axis=1
    )

    # Implied/fair probabilities + edge (so we can pick best book later)
    def implied_prob(american):
        o = _num_int(american)
        if pd.isna(o): return np.nan
        return (-o)/(-o+100.0) if o < 0 else 100.0/(o+100.0)

    dfj["p_over_imp"]  = dfj["over_odds"].apply(implied_prob)
    dfj["p_under_imp"] = dfj["under_odds"].apply(implied_prob)

    def devig_pair(p_over_imp, p_under_imp):
        if pd.isna(p_over_imp) or pd.isna(p_under_imp):
            return (np.nan, np.nan)
        s = p_over_imp + p_under_imp
        if s <= 0:
            return (np.nan, np.nan)
        return (p_over_imp/s, p_under_imp/s)

    fair = dfj.apply(
        lambda r: pd.Series(devig_pair(r["p_over_imp"], r["p_under_imp"]), index=["p_over_fair","p_under_fair"]),
        axis=1
    )
    dfj = pd.concat([dfj, fair], axis=1)

    dfj["edge_over"] = np.where(
        dfj["p_over_fair"].notna(),
        dfj["p_over_model"] - dfj["p_over_fair"],
        dfj["p_over_model"] - dfj["p_over_imp"]
    )

    # Keep the best book per player/market (highest edge)
    dfj = dfj.sort_values(["player","market","edge_over"], ascending=[True, True, False])
    dfj = dfj.drop_duplicates(subset=["player","market"], keep="first")

    joined_frames.append(
        dfj[[
            "player","team","opponent","market","line","book","game_date",
            "over_odds","under_odds","projection_mean","projection_sd",
            "projected_minutes","start_prob","lineup_status","p_over_model","edge_over"
        ]]
    )

# Combined joined frame for all markets
if not joined_frames:
    raise RuntimeError("No joined rows produced for any market.")
df_proj_join_all = pd.concat(joined_frames, ignore_index=True)

# 6) Prepare odds slice for Excel builder
df_odds_for_excel = df_proj_join_all[[
    "player","team","opponent","market","line","book","game_date","over_odds","under_odds"
]].copy()

# 7) Export full bets workbook
bets, excel_path = build_value_bets_excel(
    df_projections=df_proj_join_all,
    df_odds=df_odds_for_excel,
    outfile_path=os.path.join(FOLDER_BETS, f"value_bets_top100_{TODAY}.xlsx"),
    join_keys=("player","market")  # permissive merge
)

print(f"\nSaved value bets to: {excel_path}")
print(len(bets), "value bets found across", df_proj_join_all['market'].nunique(), "markets.")
display(bets.head(20))


Total rows in raw wide: 1872
Books present in columns: ['betrivers', 'caesars', 'draftkings', 'espnbet', 'fanduel', 'hardrock', 'mgm']
Books with lines: {'PTS': ['betrivers', 'caesars', 'draftkings', 'fanduel', 'hardrock', 'mgm'], 'REB': ['betrivers', 'caesars', 'draftkings', 'fanduel', 'hardrock', 'mgm'], 'AST': ['betrivers', 'caesars', 'draftkings', 'fanduel', 'hardrock', 'mgm']}
odds_long rows: 1868
odds_long columns: ['player', 'team', 'opponent', 'game_date', 'market', 'book', 'line', 'over_odds', 'under_odds']
     player team opponent  game_date market        book  line  over_odds  \
0  AJ Green  MIL     @IND        NaN    AST         mgm   0.5        NaN   
1  AJ Green  MIL     @IND        NaN    PTS   betrivers   9.5        NaN   
2  AJ Green  MIL     @IND        NaN    PTS     caesars   8.5        NaN   
3  AJ Green  MIL     @IND        NaN    PTS  draftkings   8.5        NaN   
4  AJ Green  MIL     @IND        NaN    PTS     fanduel   9.5        NaN   

   under_odds  
0    

,asof_date,game_date,book,player,team,opponent,market,line,lineup_status,over_odds,...,projection_mean,projection_sd,start_prob,team_odds,opponent_odds,line_odds,book_odds,game_date_odds,over_odds_odds,under_odds_odds
0,2025-11-03,NaN,betrivers,AJ Green,MIL,@IND,PTS,9.5,EXPECTED,NaN,...,7.916030,1.187405,0.9,MIL,@IND,9.5,betrivers,NaN,NaN,NaN
1,2025-11-03,NaN,betrivers,Aaron Gordon,DEN,SAC,PTS,16.5,EXPECTED,NaN,...,16.384876,2.457731,0.9,DEN,SAC,16.5,betrivers,NaN,NaN,NaN
2,2025-11-03,NaN,betrivers,Aaron Nesmith,IND,MIL,PTS,16.5,EXPECTED,NaN,...,11.439683,1.715953,0.9,IND,MIL,16.5,betrivers,NaN,NaN,NaN
3,2025-11-03,NaN,betrivers,Alex Sarr,WAS,@NYK,PTS,15.5,EXPECTED,NaN,...,13.514768,2.027215,0.9,WAS,@NYK,15.5,betrivers,NaN,NaN,NaN
4,2025-11-03,NaN,betrivers,Alperen Sengun,HOU,DAL,PTS,21.5,EXPECTED,NaN,...,7.965284,1.194793,0.9,HOU,DAL,21.5,betrivers,NaN,NaN,NaN
5,2025-11-03,NaN,betrivers,Amen Thompson,HOU,DAL,PTS,16.5,EXPECTED,NaN,...,12.298719,1.844808,0.9,HOU,DAL,16.5,betrivers,NaN,NaN,NaN
6,2025-11-03,NaN,betrivers,Andrew Wiggins,MIA,@LAC,PTS,16.5,EXPECTED,NaN,...,18.280849,2.742127,0.9,MIA,@LAC,16.5,betrivers,NaN,NaN,NaN
7,2025-11-03,NaN,betrivers,Anfernee Simons,BOS,UTA,PTS,16.5,EXPECTED,NaN,...,19.049860,2.857479,0.9,BOS,UTA,16.5,betrivers,NaN,NaN,NaN
8,2025-11-03,NaN,betrivers,Ausar Thompson,DET,@MEM,PTS,14.5,EXPECTED,NaN,...,13.016480,1.952472,0.9,DET,@MEM,14.5,betrivers,NaN,NaN,NaN
9,2025-11-03,NaN,betrivers,Bam Adebayo,MIA,@LAC,PTS,19.5,EXPECTED,NaN,...,19.007532,2.851130,0.9,MIA,@LAC,19.5,betrivers,NaN,NaN,NaN


In [26]:
# === BUILD & SAVE VALUE BETS (Top-100) TO /data/bets ===
import os, re
import numpy as np, pandas as pd
from datetime import datetime
from statistics import NormalDist

# --- ensure folder structure ---
os.makedirs("data/bets", exist_ok=True)

# ---- 0) Source table ---------------------------------------------------------
if "df_proj_join_all" in globals() and isinstance(df_proj_join_all, pd.DataFrame) and not df_proj_join_all.empty:
    df = df_proj_join_all.copy()
elif "bets" in globals() and isinstance(bets, pd.DataFrame) and not bets.empty:
    df = bets.copy()
elif "df_proj_join" in globals() and isinstance(df_proj_join, pd.DataFrame) and not df_proj_join.empty:
    df = df_proj_join.copy()
else:
    raise RuntimeError("No joined dataset found (df_proj_join_all/bets/df_proj_join). Run the join cell first.")

# ---- 1) Canonical minimal fields --------------------------------------------
def _pick(df_, names):
    for n in names:
        if n in df_.columns:
            return df_[n]
    return pd.Series([np.nan]*len(df_))

df = df.copy()
df["player"]          = _pick(df, ["player","player_proj","player_odds"])
df["team"]            = _pick(df, ["team","team_proj","team_odds"])
df["opponent"]        = _pick(df, ["opponent","opponent_proj","opponent_odds"])
df["market"]          = _pick(df, ["market"])
df["line"]            = _pick(df, ["line","posted_line","book_line"])
df["book"]            = _pick(df, ["book"])
df["over_odds"]       = _pick(df, ["over_odds"])
df["under_odds"]      = _pick(df, ["under_odds"])
df["projection_mean"] = _pick(df, ["projection_mean","expected_line"])
df["projection_sd"]   = _pick(df, ["projection_sd"])

# ---- 2) Coerce numerics ------------------------------------------------------
def _first_float(x):
    if pd.isna(x): return np.nan
    m = re.search(r"[-+]?\d+(?:\.\d+)?", str(x))
    return float(m.group()) if m else np.nan

def _first_int(x):
    if pd.isna(x): return np.nan
    m = re.search(r"[-+]?\d+", str(x))
    return float(m.group()) if m else np.nan

for col in ["line","projection_mean","projection_sd"]:
    df[col] = df[col].apply(_first_float)

# ---- 3) Model P(Over) with fallback SD --------------------------------------
if "p_over_model" not in df.columns or df["p_over_model"].isna().all():
    sd_missing = ("projection_sd" not in df.columns) or df["projection_sd"].fillna(0).eq(0).all()
    if sd_missing:
        df["projection_sd"] = (df["projection_mean"].abs() * 0.15).clip(lower=1.0)

    def p_over_from_normal(mean, sd, line):
        if pd.isna(mean) or pd.isna(sd) or pd.isna(line) or float(sd) <= 0:
            return np.nan
        z = (float(line) - float(mean)) / float(sd)
        return 1.0 - NormalDist().cdf(z)

    df["p_over_model"] = df.apply(
        lambda r: p_over_from_normal(r["projection_mean"], r["projection_sd"], r["line"]),
        axis=1
    )

# ---- 4) Implied / fair probs + edge ------------------------------------------
def implied_prob_from_american(american):
    o = _first_int(american)
    if pd.isna(o): return np.nan
    return (-o)/(-o+100.0) if o < 0 else 100.0/(o+100.0)

if "p_over_imp" not in df.columns:
    df["p_over_imp"] = df["over_odds"].apply(implied_prob_from_american)
if "p_under_imp" not in df.columns:
    df["p_under_imp"] = df["under_odds"].apply(implied_prob_from_american)

fair = df.apply(
    lambda r: pd.Series(
        (np.nan, np.nan) if (pd.isna(r["p_over_imp"]) or pd.isna(r["p_under_imp"])) else
        (r["p_over_imp"]/(r["p_over_imp"]+r["p_under_imp"]),
         r["p_under_imp"]/(r["p_over_imp"]+r["p_under_imp"]))
    , index=["p_over_fair","p_under_fair"]), axis=1)
df = pd.concat([df, fair], axis=1)

df["edge_over"] = np.where(
    df["p_over_fair"].notna(),
    df["p_over_model"] - df["p_over_fair"],
    df["p_over_model"] - df["p_over_imp"]
)

# Fallback z-score edge
nd = NormalDist()
df["z_score"] = (df["projection_mean"] - df["line"]) / df["projection_sd"].replace(0, np.nan)
df["edge_fallback"] = df["z_score"].map(lambda z: (nd.cdf(z) - 0.5)*2 if pd.notna(z) else np.nan)
df["edge_rank"] = np.where(df["edge_over"].notna(), df["edge_over"], df["edge_fallback"])

# ---- 5) Split: priced vs all -------------------------------------------------
base_cols = [
    "player","team","opponent","market","line","book",
    "over_odds","under_odds","p_over_imp","p_under_imp",
    "p_over_fair","p_under_fair",
    "projection_mean","projection_sd","p_over_model","edge_over","edge_rank"
]
lean = df.loc[:, [c for c in base_cols if c in df.columns]].copy()
lean = lean.dropna(subset=["player","market","line","projection_mean"], how="any")
lean = lean.sort_values(["player","market","edge_rank"], ascending=[True, True, False])
lean = lean.drop_duplicates(subset=["player","market"], keep="first")

priced = lean.dropna(subset=["over_odds","under_odds","p_over_imp","p_under_imp"], how="any").copy()
allp   = lean.copy()

# Expected value for $100 bet
def ev_over_100(p_over, american):
    o = _first_int(american)
    if pd.isna(p_over) or pd.isna(o): return np.nan
    dec = (1 + o/100.0) if o > 0 else (1 + 100.0/abs(o))
    return p_over * (dec - 1) * 100 - (1 - p_over) * 100

priced["EV_over_$100"] = priced.apply(lambda r: ev_over_100(r["p_over_model"], r["over_odds"]), axis=1)

# ---- 6) Write both to Excel --------------------------------------------------
out_path = os.path.join("data/bets", f"value_bets_top100_{datetime.utcnow().strftime('%Y%m%d')}.xlsx")
with pd.ExcelWriter(out_path, engine="openpyxl") as w:
    priced.to_excel(w, sheet_name="Top100_priced", index=False)
    allp.to_excel(w, sheet_name="Top100_all", index=False)

print(f"✅ Saved Top-100 value bets to: {out_path}")
print(f"Rows → priced: {len(priced)} | all: {len(allp)}")


PermissionError: [Errno 13] Permission denied: 'data/bets\\value_bets_top100_20251103.xlsx'

In [4]:
# === EVALUATE YESTERDAY'S BETS (Europe/Athens) → saves to /data/eval ===
import os, re, numpy as np, pandas as pd
from glob import glob
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

# ------------------ settings: always "day before" in Europe/Athens ------------------
TZ = ZoneInfo("Europe/Athens")
today_local = datetime.now(TZ).date()
ydate = today_local - timedelta(days=1)                 # <-- Yesterday (local)
ystr = ydate.strftime("%Y%m%d")
print(f"Evaluating bets for YESTERDAY (Europe/Athens): {ydate} ({ystr})")

os.makedirs("data/eval", exist_ok=True)

# ------------------ helpers ------------------
def _norm_player(s):
    if not isinstance(s, str): return ""
    return re.sub(r"[.`'’\-]", "", s.strip()).lower()

def pick_col(df, candidates, default=np.nan):
    for c in candidates:
        if c in df.columns:
            return df[c]
    return pd.Series([default]*len(df))

def _first_float(x):
    if pd.isna(x): return np.nan
    m = re.search(r"[-+]?\d+(?:\.\d+)?", str(x))
    return float(m.group()) if m else np.nan

def infer_opponent(df):
    if "OPPONENT_ABBREVIATION" in df.columns:
        return df["OPPONENT_ABBREVIATION"]
    matchup = pick_col(df, ["MATCHUP","Matchup"])
    team = pick_col(df, ["TEAM_ABBREVIATION","TEAM"])
    out = []
    for t, m in zip(team.fillna(""), matchup.fillna("")):
        opp = np.nan
        if isinstance(m, str) and m:
            parts = re.split(r"[@vVsS]+\.*", m)
            if len(parts) >= 2:
                cand = parts[-1].strip().upper()
                if cand == str(t).upper() and len(parts) >= 2:
                    cand = parts[0].strip().upper()
                opp = cand
        out.append(opp)
    return pd.Series(out, index=df.index)

def _parse_date_from_filename(path, pattern):
    m = re.search(pattern, os.path.basename(path))
    return m.group(1) if m else None

# ------------------ 1) Pick yesterday's bets workbook ------------------
bet_files = sorted(glob("data/bets/value_bets_top100_*.xlsx"))
if not bet_files:
    raise FileNotFoundError("No bets files found in data/bets/. Run the Top-100 export first.")

# Prefer exact yesterday; if missing, use nearest earlier
dated = []
for p in bet_files:
    ds = _parse_date_from_filename(p, r"value_bets_top100_(\d{8})\.xlsx")
    if ds:
        dated.append((ds, p))
dated = sorted(dated, key=lambda x: x[0])

bets_path = None
for ds, p in reversed(dated):
    if ds <= ystr:
        bets_path = p
        break
if bets_path is None:
    # fall back to earliest (shouldn't usually happen)
    bets_path = dated[0][1]
    print("⚠️ No bets file on/before yesterday; using earliest available:", os.path.basename(bets_path))
else:
    print("Using bets workbook:", os.path.basename(bets_path))

# ------------------ 2) Load a non-empty Top100 sheet ------------------
with pd.ExcelFile(bets_path) as xf:
    sheet = None
    for s in ["Top100_priced","Top100_all","Top100"]:
        if s in xf.sheet_names:
            tmp = pd.read_excel(bets_path, sheet_name=s)
            if not tmp.empty:
                sheet, bets = s, tmp
                break
if sheet is None:
    raise RuntimeError("All Top100 sheets empty in bets workbook.")

print(f"Loaded bets: sheet [{sheet}], rows={len(bets)}")

# ------------------ 3) Load boxscores for YESTERDAY ------------------
# Priority: in-memory `box_d` filtered to ydate → file nba_boxscores_YYYYMMDD.csv → latest fallback
def _as_date(s):
    try:
        return pd.to_datetime(s).date()
    except Exception:
        return None

box = None
if "box_d" in globals() and isinstance(box_d, pd.DataFrame) and not box_d.empty:
    bd = box_d.copy()
    # Try to locate a date column and filter to yesterday
    date_col = None
    for c in ["GAME_DATE", "GAME_DATE_EST", "GAME_DATE_LCL", "Date", "date"]:
        if c in bd.columns:
            date_col = c
            break
    if date_col is not None:
        bd["_gdate"] = bd[date_col].apply(_as_date)
        box = bd.loc[bd["_gdate"].eq(ydate)].copy()
        print(f"box_d in memory → filtered rows for {ydate}: {len(box)}")
    else:
        print("⚠️ box_d found but no date column to filter; using all rows.")
        box = bd.copy()

if box is None or box.empty:
    # Try file with exact date
    exact_csv = f"nba_boxscores_{ystr}.csv"
    if os.path.exists(exact_csv):
        box = pd.read_csv(exact_csv)
        print(f"Loaded boxscores from file: {exact_csv} ({len(box)} rows)")
    else:
        # pick latest available matching pattern
        csv_files = sorted(glob("nba_boxscores_*.csv"))
        if csv_files:
            box = pd.read_csv(csv_files[-1])
            print(f"⚠️ No boxscore file for {ystr}; using latest: {os.path.basename(csv_files[-1])} ({len(box)} rows)")
        else:
            raise FileNotFoundError("No boxscore CSV found (nba_boxscores_YYYYMMDD.csv).")

# ------------------ 4) Normalize & (optionally) filter box by yesterday ------------------
# Try filter again if a date column exists (helps when we loaded a combined CSV)
for c in ["GAME_DATE", "GAME_DATE_EST", "GAME_DATE_LCL", "Date", "date"]:
    if c in box.columns:
        _dates = pd.to_datetime(box[c], errors="coerce").dt.date
        if _dates.notna().any():
            box = box.loc[_dates.eq(ydate)].copy()
            print(f"Filtered boxscores to {ydate} by column '{c}': {len(box)} rows")
        break

# ------------------ 5) Prepare bets & box for join ------------------
bets = bets.copy()
bets["player"] = pick_col(bets, ["player","Player"])
bets["market"] = pick_col(bets, ["market","Market"])
bets["line"]   = pd.to_numeric(pick_col(bets, ["posted_line","line"]), errors="coerce")
bets["player_key"] = bets["player"].map(_norm_player)

box = box.copy()
box["player"] = pick_col(box, ["PLAYER_NAME","Player"])
box["player_key"] = box["player"].map(_norm_player)
box["PTS"] = pd.to_numeric(pick_col(box, ["PTS","Points"]), errors="coerce")
box["REB"] = pd.to_numeric(pick_col(box, ["REB","Rebounds"]), errors="coerce")
box["AST"] = pd.to_numeric(pick_col(box, ["AST","Assists"]), errors="coerce")
box["TEAM_ABBREVIATION"] = pick_col(box, ["TEAM_ABBREVIATION","TEAM"])
box["OPPONENT_ABBREVIATION"] = infer_opponent(box)

# ------------------ 6) Join & grade ------------------
joined = bets.merge(
    box[["player_key","PTS","REB","AST","TEAM_ABBREVIATION","OPPONENT_ABBREVIATION"]],
    on="player_key", how="left", suffixes=("","_box")
)

def pick_actual(row):
    m = str(row.get("market","")).upper()
    return row.get(m, np.nan) if m in ["PTS","REB","AST"] else np.nan

joined["actual"] = joined.apply(pick_actual, axis=1)
joined["result_over"] = np.where(
    joined["actual"].notna() & joined["line"].notna(),
    np.where(joined["actual"] > joined["line"], "WIN",
    np.where(joined["actual"] == joined["line"], "PUSH", "LOSS")),
    "NA"
)

# ------------------ 7) Summary ------------------
graded = joined["result_over"].isin(["WIN","LOSS","PUSH"]).sum()
wins = (joined["result_over"]=="WIN").sum()
losses = (joined["result_over"]=="LOSS").sum()
pushes = (joined["result_over"]=="PUSH").sum()
hitrate = wins / max(wins+losses, 1)

print(f"\n✅ Evaluated {graded} bets for {ydate}  (WIN={wins}, LOSS={losses}, PUSH={pushes})")
print(f"🎯 Hit rate: {hitrate:.1%}")

# ------------------ 8) Save evaluation ------------------
eval_out = os.path.join("xgb_outputs/evaluations", f"value_bets_top100_{ystr}_EVAL.csv")
joined.to_csv(eval_out, index=False)
print(f"📊 Saved evaluation to: {eval_out}")

display(joined.head(25))


Evaluating bets for YESTERDAY (Europe/Athens): 2025-11-03 (20251103)
Using bets workbook: value_bets_top100_20251103.xlsx
Loaded bets: sheet [Top100_all], rows=377
⚠️ No boxscore file for 20251103; using latest: nba_boxscores_2025-26.csv (2313 rows)
Filtered boxscores to 2025-11-03 by column 'GAME_DATE': 198 rows

✅ Evaluated 353 bets for 2025-11-03  (WIN=180, LOSS=173, PUSH=0)
🎯 Hit rate: 51.0%
📊 Saved evaluation to: xgb_outputs/evaluations\value_bets_top100_20251103_EVAL.csv


,player,team,opponent,market,line,book,over_odds,under_odds,p_over_imp,p_under_imp,...,edge_over,edge_rank,player_key,PTS,REB,AST,TEAM_ABBREVIATION,OPPONENT_ABBREVIATION,actual,result_over
0,Isaiah Collier,UTA,@BOS,PTS,6.5,caesars,NaN,NaN,NaN,NaN,...,NaN,0.999799,isaiah collier,NaN,NaN,NaN,NaN,NaN,NaN,NA
1,Bub Carrington,WAS,@NYK,AST,3.5,caesars,NaN,NaN,NaN,NaN,...,NaN,0.999731,bub carrington,0.0,4.0,7.0,WAS,NYK,7.0,WIN
2,Brook Lopez,LAC,MIA,PTS,6.5,caesars,NaN,NaN,NaN,NaN,...,NaN,0.999482,brook lopez,5.0,1.0,0.0,LAC,MIA,5.0,LOSS
3,Isaiah Collier,UTA,@BOS,AST,4.5,caesars,NaN,NaN,NaN,NaN,...,NaN,0.999007,isaiah collier,NaN,NaN,NaN,NaN,NaN,NaN,NA
4,Kyle Filipowski,UTA,@BOS,PTS,6.5,mgm,NaN,NaN,NaN,NaN,...,NaN,0.998622,kyle filipowski,13.0,8.0,0.0,UTA,,13.0,WIN
5,Cameron Johnson,DEN,SAC,AST,2.5,betrivers,NaN,NaN,NaN,NaN,...,NaN,0.997917,cameron johnson,10.0,3.0,2.0,DEN,AC,2.0,LOSS
6,Jalen Wilson,BKN,MIN,PTS,5.5,mgm,NaN,NaN,NaN,NaN,...,NaN,0.997296,jalen wilson,7.0,1.0,1.0,BKN,MIN,7.0,WIN
7,Kyle Kuzma,MIL,@IND,PTS,8.5,draftkings,NaN,NaN,NaN,NaN,...,NaN,0.994089,kyle kuzma,15.0,5.0,0.0,MIL,IND,15.0,WIN
8,Shaedon Sharpe,POR,LAL,REB,3.5,betrivers,NaN,NaN,NaN,NaN,...,NaN,0.993276,shaedon sharpe,23.0,7.0,2.0,POR,LAL,7.0,WIN
9,Kel'el Ware,MIA,@LAC,PTS,7.5,betrivers,NaN,NaN,NaN,NaN,...,NaN,0.989611,kelel ware,16.0,5.0,0.0,MIA,LAC,16.0,WIN
